In [16]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import torchvision as T
import torch
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import math
import itertools
import os
import sys
root_path = os.path.abspath(os.path.join('../..')) # <- adjust such that root_path always points at the root project dir (i.e. if current file is two folders deep, use '../..'). 
if root_path not in sys.path:
    sys.path.append(root_path)
import database_server.db_utilities as dbu 
import pickle as pkl
from sklearn.decomposition import PCA
from Help_functions import preprocess, game_dict, inputs, club_dict, points_and_co, points_and_co_oppon, data_to_lstm, Sport_pred_2LSTM_1, predict
import random

In [17]:
torch.manual_seed(420)
random.seed(420)
np.random.seed(420)

In [18]:
query_str = """
SELECT ms.*, 
       m.schedule_date, m.schedule_time, m.schedule_round, m.schedule_day,
       w.annual_wages_eur AS annual_wage_team, 
       w.weekly_wages_eur AS weekly_wages_eur,
       w.annual_wages_eur/w.n_players AS annual_wage_player_avg
FROM matchstats ms 
LEFT JOIN matches m ON ms.match_id = m.id
LEFT JOIN teamwages w ON ms.team_id = w.team_id
AND     ms.season_str = w.season_str
ORDER BY m.schedule_date DESC, m.schedule_time DESC; 
"""

df_allinfo = dbu.select_query(query_str)

In [19]:
new_data_test = preprocess(df_allinfo)

In [20]:
scale_df = new_data_test.data_frame.copy()
#scale_df["stad_capac"] = 0
for team in scale_df.team_id.unique():
    max_attend = max(scale_df[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"])].attendance)
    scale_df.loc[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"]),"stad_capac"] = scale_df[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"])].attendance.apply(lambda x: x/max_attend)
    scale_df.loc[(scale_df.opponent_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Away"]),"stad_capac"] = scale_df[(scale_df.opponent_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Away"])].attendance.apply(lambda x: x/max_attend)

In [21]:
ohe_team = OneHotEncoder()
to_ohe_team = scale_df.loc[:, ["team_id", "opponent_id"]]
ohe_team.fit(to_ohe_team)

codes = ohe_team.transform(to_ohe_team).toarray()
feature_names = ohe_team.get_feature_names(['team_id', 'opponent_id'])

scale_df = pd.concat([scale_df, 
               pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)
##########################################
ohe_ligue = OneHotEncoder()
to_ohe_ligue = scale_df.loc[:,["league_id"]]
ohe_ligue.fit(to_ohe_ligue)

codes = ohe_ligue.transform(to_ohe_ligue).toarray()
feature_names = ohe_ligue.get_feature_names(['league_id'])

scale_df = pd.concat([scale_df, 
               pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)

In [22]:
clubs = club_dict(scale_df)

In [23]:
result_dict = new_data_test.return_dicts("result")
clubs = points_and_co(clubs, result_dict)

In [24]:
clubs = points_and_co_oppon(clubs, result_dict)

In [25]:
game_train, game_valid = game_dict(scale_df)

In [26]:
abcdefg = list(scale_df.columns)
abc = abcdefg[:abcdefg.index("annual_wage_player_avg")+1]
defg = abcdefg[abcdefg.index("annual_wage_player_avg")+1:]

rearange_list = ['result', 'gf', 'ga', 'goal_diff', 'xg', 'xga', 'shooting_standard_gls',
       'shooting_standard_sh', 'shooting_standard_sot',
       'shooting_standard_sot_perc', 'shooting_standard_g_per_sh',
       'shooting_standard_g_per_sot', 'shooting_standard_dist',
       'shooting_standard_fk', 'shooting_standard_pk',
       'shooting_standard_pkatt', 'shooting_expected_npxg',
       'shooting_expected_npxg_per_sh', 'shooting_expected_g_minus_xg',
       'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
       'keeper_performance_saves', 'keeper_performance_save_perc',
       'keeper_performance_cs', 'keeper_performance_psxg',
       'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt',
       'keeper_penaltykicks_pka', 'keeper_penaltykicks_pksv',
       'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
       'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr',
       'keeper_passes_launch_perc', 'keeper_passes_avglen',
       'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc',
       'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp',
       'keeper_crosses_stp_perc', 'keeper_sweeper_number_opa',
       'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
       'passing_total_cmp_perc', 'passing_total_totdist',
       'passing_total_prgdist', 'passing_short_cmp', 'passing_short_att',
       'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
       'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att',
       'passing_long_cmp_perc', 'passing_attacking_ast',
       'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
       'passing_attacking_1_per_3', 'passing_attacking_ppa',
       'passing_attacking_crspa', 'passing_attacking_prgp',
       'passing_types_passtypes_live', 'passing_types_passtypes_dead',
       'passing_types_passtypes_fk', 'passing_types_passtypes_tb',
       'misc_aerialduels_won_perc','attendance', 'points', 'mean_points',
       'weekly_wages_eur', 'season_str',  'league_id', 'venue', 'team_id',
       'opponent_id', 'last_results', 'oppon_points', 'oppon_mean_points', 'schedule_round',
        'captain', 'formation', 'referee',  'match_id', 'schedule_date', 'schedule_time',
        'schedule_day', 'annual_wage_team', 'annual_wage_player_avg',]

rearange_list = list(itertools.chain.from_iterable(defg if item == "team_id" else [item] for item in rearange_list))

del rearange_list[rearange_list.index("opponent_id")]
rearange_list

['result',
 'gf',
 'ga',
 'goal_diff',
 'xg',
 'xga',
 'shooting_standard_gls',
 'shooting_standard_sh',
 'shooting_standard_sot',
 'shooting_standard_sot_perc',
 'shooting_standard_g_per_sh',
 'shooting_standard_g_per_sot',
 'shooting_standard_dist',
 'shooting_standard_fk',
 'shooting_standard_pk',
 'shooting_standard_pkatt',
 'shooting_expected_npxg',
 'shooting_expected_npxg_per_sh',
 'shooting_expected_g_minus_xg',
 'shooting_expected_npg_minus_xg',
 'keeper_performance_sota',
 'keeper_performance_saves',
 'keeper_performance_save_perc',
 'keeper_performance_cs',
 'keeper_performance_psxg',
 'keeper_performance_psxg_plus_minus',
 'keeper_penaltykicks_pkatt',
 'keeper_penaltykicks_pka',
 'keeper_penaltykicks_pksv',
 'keeper_penaltykicks_pkm',
 'keeper_launched_cmp',
 'keeper_launched_att',
 'keeper_launched_cmp_perc',
 'keeper_passes_att',
 'keeper_passes_thr',
 'keeper_passes_launch_perc',
 'keeper_passes_avglen',
 'keeper_goalkicks_att',
 'keeper_goalkicks_launch_perc',
 'keeper_

In [27]:
train_inputs = inputs(game_train, clubs, rearange_list, scale_df)
valid_inputs = inputs(game_valid, clubs, rearange_list, scale_df)

7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [28]:
train_for_loader = data_to_lstm(train_inputs)
valid_for_loader = data_to_lstm(valid_inputs)

In [29]:
train_loader = torch.utils.data.DataLoader(train_for_loader, batch_size = 32, drop_last = True)
test_loader = torch.utils.data.DataLoader(valid_for_loader, batch_size = 32, drop_last = True)

In [30]:
len_input = train_inputs[0][0].shape[1]
lr = 2e-5
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    accur += pred.eq(result).sum().item()

    pred = net(input2)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result2 = result2[:,-1,:]
    result2 = torch.argmax(result2, dim = 1)
    accur += pred.eq(result2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 64)}%")

for epoch in range(15):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1)
        result1 = result1[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result1).item()
        pred = torch.argmax(pred, dim = 1)
        result1 = torch.argmax(result1, dim = 1)
        accur += pred.eq(result1).sum().item()

        pred = net(input2)
        result2 = result2[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result2).item()
        pred = torch.argmax(pred, dim = 1)
        result2  = torch.argmax(result2, dim = 1)
        accur += pred.eq(result2).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 64))
    os.makedirs(os.path.dirname(f"./models/sequence_model/LSTM/{lr}/accur_{round(accuracy, 2)}"), exist_ok = True)
    torch.save(net.state_dict(), f"./models/sequence_model/LSTM/{lr}/accur_{round(accuracy, 2)}")
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


Loss accuracy training: 36.95652173913044%
Epoch: 0
Loss 40.608262330293655 accuracy 42.49320652173913%
Epoch: 1
Loss 40.53224788606167 accuracy 42.08559782608695%
Epoch: 2
Loss 40.07095202803612 accuracy 42.56114130434783%
Epoch: 3
Loss 40.33928169310093 accuracy 42.59510869565217%
Epoch: 4
Loss 40.06759561598301 accuracy 42.255434782608695%
Epoch: 5
Loss 40.065742045640945 accuracy 42.79891304347826%
Epoch: 6
Loss 40.19216784834862 accuracy 42.900815217391305%
Epoch: 7
Loss 39.845893770456314 accuracy 42.79891304347826%
Epoch: 8
Loss 39.964572086930275 accuracy 42.66304347826087%
Epoch: 9


[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [ ]:
len_input = train_inputs[0][0].shape[1]
lr = 2e-5
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    pred2 = net(input2)
    predicted = predict(pred1, pred2, result_dict)
    accur += predicted.eq(result).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

for epoch in range(3):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred1 = net(input1)
        pred2 = net(input2)
        result  = torch.argmax(result1, dim = 1)
        predicted = torch.nn.functional.one_hot(predict(pred1, pred2, result_dict), num_classes = 3)
        accur += predicted.eq(result).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 32))
    os.makedirs(os.path.dirname(f"./models/sequence_model/LSTM/{lr}/accur_{round(accuracy, 2)}"), exist_ok = True)
    torch.save(net.state_dict(), f"./models/sequence_model/LSTM/{lr}/accur_{round(accuracy, 2)}")
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


In [31]:
new_data_test.return_dicts("venue")

{'Home': 0, 'Away': 1}

In [ ]:
data = pd.read_csv("/Users/paul/Documents/Uni/SoSe23/dsp2/DS-Project/data/scraped/cleaned/match/data_match_clean.csv", delimiter = ";", index_col = 1)
data = data.drop(["Unnamed: 0"], axis = 1)
data.head()

In [ ]:
class preprocess():
    
    def __init__(self, data_frame):
        self.data_frame = data_frame
        
        self.data_frame = self.data_frame.loc[:,~self.data_frame.columns.duplicated()].copy()
        
        self.venue_dict = {}
        i = 0
        for where in self.data_frame.venue.unique():
            self.venue_dict[where] = i
            i += 1
        
        self.day_dict = {}
        i = 0
        for day in self.data_frame.schedule_day.unique():
            self.day_dict[day] = i
            i += 1
        
        self.result = {}
        i = 0
        for res in self.data_frame.result.unique():
            self.result[res] = i
            i += 1
        #print(self.result)
        
        self.capt = {}
        i = 0
        for cap in set(list(self.data_frame.captain) + list(self.data_frame.captain)): #change
            self.capt[cap] = i
            i += 1
            
        self.formation = {}
        i = 0
        for form in self.data_frame.formation.unique(): #changed
            self.formation[form] = i
            i += 1
            
        self.referee = {}
        i = 0
        for ref in self.data_frame.referee.unique():
            self.referee[ref] = i
            i += 1
        
#        self.teams = {}
 #       i = 0
  #      for team in self.data_frame.team_id.unique():#set(list(self.data_frame.team_id) + list(self.data_frame.team_id)):
   #         self.teams[team] = i
    #        i += 1
        #print(self.teams)
        
        self.season = {}
        i = 0
        for seas in self.data_frame.season_str.unique():
            self.season[seas] = i
            i += 1
        
        self.time = {}
        i = 0
        for tim in self.data_frame.schedule_time.unique():
            self.time[tim] = i
            i += 1
            
        self.ligue = {}
        i = 0
        for lig in self.data_frame.league_id.unique():
            self.ligue[lig] = i
            i += 1
            
#        self.match = {}
 #       i = 0
  #      for mat in self.data_frame.match_id.unique():
   #         self.match[mat] = i
    #        i += 1
        
        i = 0
        self.matchweek = {}
        self.data_frame.drop(self.data_frame[self.data_frame.schedule_round == "Relegation tie-breaker"].index, inplace = True)
        self.data_frame.reset_index(drop = True, inplace = True)
        for week in self.data_frame.schedule_round.unique():
            self.matchweek[week] = int(week.split()[1])
            i += 1
        
        #self.data_frame.drop(columns = "fbref_id", inplace = True)
        
        
        self.data_frame.schedule_day.replace(self.day_dict, inplace = True)
        self.data_frame.result.replace(self.result, inplace = True) #change
        #self.data_frame.result_away.replace(self.result, inplace = True)
        self.data_frame.captain.replace(self.capt, inplace = True)
        #self.data_frame.captain_away.replace(self.capt, inplace = True)
        self.data_frame.formation.replace(self.formation, inplace = True)
        #self.data_frame.formation_away.replace(self.formation, inplace = True)
        self.data_frame.referee.replace(self.referee, inplace = True)
        #self.data_frame.team_id.replace(self.teams, inplace = True)
        #self.data_frame.team_id.replace(self.teams, inplace = True)
        self.data_frame.season_str.replace(self.season, inplace = True)
        self.data_frame.schedule_time.replace(self.time, inplace = True)
        self.data_frame.league_id.replace(self.ligue, inplace = True)
        #self.data_frame.match_id.replace(self.match, inplace = True)
        self.data_frame.schedule_round.replace(self.matchweek, inplace = True)
        self.data_frame.venue.replace(self.venue_dict, inplace = True)
        
        self.features = ['shooting_standard_gls', 'shooting_standard_sh', 'shooting_standard_sot', 
                         'shooting_standard_sot_perc', 'shooting_standard_g_per_sh', 'shooting_standard_g_per_sot',
                         'shooting_standard_dist', 'shooting_standard_fk', 'shooting_standard_pk', 
                         'shooting_standard_pkatt', 'shooting_expected_npxg', 'shooting_expected_npxg_per_sh',
                         'shooting_expected_g_minus_xg', 'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
                         'keeper_performance_saves', 'keeper_performance_save_perc', 'keeper_performance_cs',
                         'keeper_performance_psxg', 'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt', 
                         'keeper_penaltykicks_pksv', 'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
                         'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr', 'keeper_passes_launch_perc',
                         'keeper_passes_avglen', 'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc', 
                         'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp', 'keeper_crosses_stp_perc',
                         'keeper_sweeper_number_opa', 'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
                         'passing_total_cmp_perc', 'passing_total_totdist', 'passing_total_prgdist', 'passing_short_cmp',
                         'passing_short_att', 'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
                         'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att', 'passing_long_cmp_perc',
                         'passing_attacking_ast', 'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
                         'passing_attacking_1_per_3', 'passing_attacking_ppa', 'passing_attacking_crspa', 'passing_attacking_prgp',
                         'passing_types_passtypes_live', 'passing_types_passtypes_dead', 'passing_types_passtypes_fk',
                         'passing_types_passtypes_tb', 'passing_types_passtypes_sw', 'passing_types_passtypes_crs',
                         'passing_types_passtypes_ti', 'passing_types_passtypes_ck', 'passing_types_cornerkicks_in',
                         'passing_types_cornerkicks_out', 'passing_types_cornerkicks_str', 'passing_types_outcomes_off',
                         'passing_types_outcomes_blocks', 'gca_scatypes_sca', 'gca_scatypes_passlive', 'gca_scatypes_passdead',
                         'gca_scatypes_to', 'gca_scatypes_sh', 'gca_scatypes_fld', 'gca_scatypes_def', 'gca_gcatypes_gca', 
                         'gca_gcatypes_passlive', 'gca_gcatypes_passdead', 'gca_gcatypes_to', 'gca_gcatypes_sh',
                         'gca_gcatypes_fld',  'gca_gcatypes_def', 'defense_tackles_tkl', 'defense_tackles_tklw',
                         'defense_tackles_def3rd', 'defense_tackles_mid3rd', 'defense_tackles_att3rd', 'defense_challenges_tkl',
                         'defense_challenges_att', 'defense_challenges_tkl_perc', 'defense_challenges_lost', 'defense_blocks_blocks',
                         'defense_blocks_sh', 'defense_blocks_pass', 'defense_general_int', 'defense_general_tkl_plus_int',
                         'defense_general_clr', 'defense_general_err', 'possession_general_poss', 'possession_touches_touches',
                         'possession_touches_defpen', 'possession_touches_def3rd', 'possession_touches_mid3rd',
                         'possession_touches_att3rd', 'possession_touches_attpen', 'possession_touches_live', 'possession_takeons_att',
                         'possession_takeons_succ', 'possession_takeons_succ_perc', 'possession_takeons_tkld', 'possession_takeons_tkld_perc',
                         'possession_carries_carries', 'possession_carries_totdist', 'possession_carries_prgdist',
                         'possession_carries_prgc', 'possession_carries_1_per_3', 'possession_carries_cpa', 'possession_carries_mis',
                         'possession_carries_dis', 'possession_receiving_rec', 'possession_receiving_prgr',
                         'misc_performance_crdy', 'misc_performance_crdr', 'misc_performance_2crdy', 'misc_performance_fls',
                         'misc_performance_fld', 'misc_performance_off', 'misc_performance_og', 'misc_performance_recov',
                         'misc_aerialduels_won', 'misc_aerialduels_lost', 'misc_aerialduels_won_perc', 'keeper_penaltykicks_pka',
                         'attendance']
        
        
        #self.data_frame = data_frame
    
    def data_frame(self):
        return self.data_frame
        
    def dataset_team(self, team_id):
        # split dataset 
        random_team1 = self.data_frame[self.data_frame.fbref_home_id == team_id].loc[:, self.data_frame.columns.str.endswith('home')]
        random_team2 = self.data_frame[self.data_frame.fbref_away_id == team_id].loc[:, self.data_frame.columns.str.endswith('away')]
        random_team3 = self.data_frame[(self.data_frame.fbref_home_id == team_id) |
                        (self.data_frame.fbref_away_id == team_id)].loc[:,['schedule_time', 'attendance',
                                                                           'referee', 'fbref_season',
                                                                           'fbref_league_id', 'fbref_home_id',
                                                                           'fbref_away_id', 'fbref_match_id']]

        #random_team2.rename({"gf_away": "gf_home", "gf_home": "gf_away"}, axis = 1, inplace = True)
        # preprocess chunks
        self.concat_preprocess(random_team1, "home")
        self.concat_preprocess(random_team2, "away")
        random_team = pd.concat([random_team1, random_team2])

        # dummy to account for home and away
        random_team3["home"] = np.where(random_team3.fbref_home_id == team_id, 1, 0)

        # rename columns to change them
        random_team3_away = random_team3.loc[random_team3.fbref_away_id == team_id].copy()
        random_team3_away.rename({"fbref_home_id": "fbref_away_id",
                                  "fbref_away_id": "fbref_home_id"}, axis = 1, inplace = True)
        random_team3_home = random_team3[random_team3.fbref_home_id == team_id]

        # concat after changing column names and rename 
        random_team3 = pd.concat([random_team3_home, random_team3_away])
        random_team3.rename({"fbref_home_id": "fbref_own", 
                             "fbref_away_id": "fbref_oppon",
                             "gf_home": "gf_own", 
                             "gf_away": "gf_own"}, axis = 1, inplace = True)

        self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()

        return self.end_team

    def concat_preprocess(self, df, where):
        colname_dict = {}
        result_dict = {self.result["W"]: "L", self.result["D"]: "D", self.result["L"]: "W"}
        for colname in self.data_frame.loc[:, self.data_frame.columns.str.endswith(where)]:
            colname_dict[colname]  = colname[:-5]
    
        return df.rename(colname_dict, axis = 1, inplace = True)
    
    def return_dicts(self, dict_name):
        dicts = {"day": self.day_dict,
                 "result": self.result,
                 "captain": self.capt,
                 "formation": self.formation,
                 "referee": self.referee,
                 "teams": self.teams,
                 "season": self.season}
        return dicts[dict_name]
        if dict_name == "day":
            return self.day_dict
        elif dict_name == "result":
            return self.result
        elif dict_name == "captain":
            return self.capt
        elif dict_name == "formation":
            return self.formation
        elif dict_name == "referee":
            return self.referee
        elif dict_name == "teams":
            return self.teams
        elif dict_name == "season":
            return self.season
        
    def data_for_team1(self, date, team_1, window_size = 10, discount = 0.5):
        
        object = StandardScaler()
        dataset1 = self.dataset_team(team_1)
        cols_for_movavg = dataset1.columns.isin(self.features)
        dataset1.loc[:,cols_for_movavg] = self.moving_average(dataset1.loc[:,cols_for_movavg], window_size, discount)
        dataset1 = dataset1.add_suffix('_1')
        dataset1 = dataset1.loc[date]
        
        dataset2 = self.dataset_team(dataset1.fbref_oppon_1)
        dataset2.loc[:,cols_for_movavg] = self.moving_average(dataset2.loc[:,cols_for_movavg], window_size, discount)
        dataset2 = dataset2.add_suffix('_2')
        dataset2 = dataset2.loc[date]
        
        cols_to_drop = ['gf_1', 'xg_1', 'schedule_time_1', 'referee_1', 'fbref_season_1', 'fbref_league_id_1', 
                'fbref_own_1', 'fbref_oppon_1', 'fbref_match_id_1','gf_2', 'xg_2', 
                'fbref_own_2', 'fbref_oppon_2', 'fbref_match_id_2']
        
        to_return = pd.concat([dataset1, dataset2])#.loc[date]
        to_return = to_return[~to_return.index.isin(cols_to_drop)]
        
        X = to_return[~to_return.index.isin(["result_1", "result_2"])]
        y = to_return[to_return.index.isin(["result_1"])]
        
        return X, y
    
    def moving_average(self, df, window_size = 10, discount = 0.5, date = None):
        "compute the moving average with recency bias"
        weights = np.array([discount**x for x in reversed(range(window_size))]) # discount y_t-windowsize with discount**windowsize
        weights = np.append(weights , 0)
        sum_weights = np.sum(weights)
        mov_avg = df.rolling(window = window_size + 1).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False) # compute moving average of window_size 
        return mov_avg
        

In [ ]:
query_str = """
SELECT ms.*, 
       m.schedule_date, m.schedule_time, m.schedule_round, m.schedule_day,
       w.annual_wages_eur AS annual_wage_team, 
       w.weekly_wages_eur AS weekly_wages_eur,
       w.annual_wages_eur/w.n_players AS annual_wage_player_avg
FROM matchstats ms 
LEFT JOIN matches m ON ms.match_id = m.id
LEFT JOIN teamwages w ON ms.team_id = w.team_id
AND     ms.season_str = w.season_str
ORDER BY m.schedule_date DESC, m.schedule_time DESC; 
"""

df_allinfo = dbu.select_query(query_str)

In [ ]:
new_data_test = preprocess(df_allinfo)

In [ ]:
scale_df = new_data_test.data_frame.copy()

liste = ['schedule_time', 'schedule_round', 'schedule_day', 'result', 'gf', 'ga', 'xg', 'xga', 'formation', 
         'referee', 'season_str', 'league_id', 'team_id', 'opponent_id', 'match_id', 'id', 'fbref_id', 
         'home_team_id', 'away_team_id', 'schedule_date', 'venue', 'captain',]

cols_to_scale = list(set(list(scale_df.columns)).difference(liste))
object = StandardScaler()

scale_df.loc[:,cols_to_scale] = object.fit_transform(scale_df.loc[:,cols_to_scale])
scale_df.head()

In [ ]:
def game_dict(scale_df):
    games_train = {}
    for seas in scale_df.season_str.unique()[:-1]:
        for week in sorted(scale_df.schedule_round.unique()):
            if week not in games_train.keys():
                games_train[week] = []
            df = scale_df[(scale_df.season_str == seas) & (scale_df.schedule_round == week)]
            df1 = df[df.venue == 0]
            df2 = df[df.venue == 1]
            df = df1.merge(df2, on = "match_id")
            print(df.columns)
            break
            for game in df.match_id:
                games_train[week].append(game)
        break
    
    games_test = {}
    seas = scale_df.season_str.unique()[-1]
    for week in sorted(scale_df.schedule_round.unique()):
        if week not in games_test.keys():
            games_test[week] = []
        df = scale_df[(scale_df.season_str == seas) & (scale_df.schedule_round == week)]
        df1 = df[df.venue == 0]
        df2 = df[df.venue == 1]
        df = df1.merge(df2, on = "match_id")
        for game in df.match_id:
            games_test[week].append(game)
                
    return games_train, games_test

In [ ]:
game_train, game_valid = game_dict(scale_df)

In [ ]:
x = scale_df.loc[:,"shooting_standard_gls":'misc_aerialduels_won_perc'].fillna(0)

pca = PCA().fit(x)
a = np.cumsum(pca.explained_variance_ratio_)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
i = np.where(np.isclose(a, 0.95, atol=0.001))

In [ ]:
pca_matchstat = PCA(n_components = 0.95)
pcs_matchstat = pca_matchstat.fit_transform(x)

In [ ]:
principal_ms_df = pd.DataFrame(data = pcs_matchstat, columns = [f"feature_{p}" for p in range(int(i[0]))])
principal_ms_df.head()

In [ ]:
columns_to_overwrite = list(scale_df.loc[:,"shooting_standard_gls":'misc_aerialduels_won_perc'].columns)
scale_df_pca = scale_df

scale_df_pca.drop(labels = columns_to_overwrite, axis = "columns", inplace = True)
scale_df_pca = original_df[columns_to_overwrite] = other_data_frame[columns_to_overwrite]

In [ ]:
def save_object(obj, save_name):
        """
        Saves obj to a pickle file in directory path.
        """ 
        with open(os.path.join(f"{save_name}.pkl"), 'wb') as f:
            pkl.dump(obj, f)

def load_data_prep_object(obj_name):
    """
    Loads and returns the specified data prep object from the data_prep_objects_path.
    """
        # check if file exists
    if not os.path.isfile(os.path.join(f"{obj_name}.pkl")):
        raise ValueError(f"Data prep object file '{obj_name}.pkl' does not exist.")
    else:
        with open(os.path.join(f"{obj_name}.pkl"), 'rb') as f:
            obj = pkl.load(f)
            return obj
            
def do_pca(df, perc_var, col_name1 = "shooting_standard_gls", col_name2 = 'misc_aerialduels_won_perc'):
    try:
        pcs_matchstat = load_data_prep_object("./pca_matchstats")
    except:
        x = df.loc[:,col_name1:col_name2].fillna(0)
        pca_matchstat = PCA(n_components = perc_var)
        pcs_matchstat = pca_matchstat.fit_transform(x)
        save_object(pcs_matchstat, "pca_matchstats")
    principal_ms_df = pd.DataFrame(data = pcs_matchstat, columns = [f"feature_{p}" for p in range(pcs_matchstat.shape[1])])
    num_pcs = principal_ms_df.shape[1]
    print(num_pcs)
    columns_to_overwrite = list(df.loc[:, col_name1:col_name2].columns)
    df = df.drop(labels = columns_to_overwrite, axis = "columns")
    new_cols = columns_to_overwrite[:num_pcs-1] + columns_to_overwrite[-1:]
    print(len(new_cols))
    df.loc[:, new_cols] = principal_ms_df.values
    return df

In [ ]:
scale_df_pca = do_pca(scale_df, 0.95)

In [ ]:
scale_df_pca.head()

In [ ]:
scale_df.shape

In [ ]:
def club_dict(scale_df):
    clubs = {}
    for club in list(scale_df.team_id.unique()):
        club_df = scale_df[scale_df.team_id == club]
        club_df.sort_values(by = ['season_str', 'schedule_round'], inplace = True)
        club_df.reset_index(drop = True, inplace = True)
        clubs[club] = club_df

In [ ]:
def points_and_co(clubs, resutlt_dict):
    result_dict = new_data_test.return_dicts("result")
    point_dict = {result_dict['W']: 3, result_dict['L']: 0, result_dict['D']: 1}
    for club in clubs:
        #print(clubs[club])
        new_column_points = []
        new_column_meanpoints = []
        last_year_game_res = {}
        last_year_game = []
        goal_diff_seasons = []
        z = 0
        for season in clubs[club].groupby("season_str"):
            season = season[1]
            new_col_seas_p = [0] * len(season)
            new_col_seas_mp = [0] * len(season)
            new_col_last_year_game = [0] * len(season)
            index = 0
            #print(season)
            last_year_game_res[z] = {}
            for _, matchday in season.iterrows():
                goal_diff_seasons.append(int(matchday.gf) - int(matchday.ga))
                if index > 0:
                    new_col_seas_p[index] = new_col_seas_p[index - 1] + point_dict[matchday.result]
                    new_col_seas_mp[index] = (new_col_seas_mp[index - 1] + point_dict[matchday.result])/2
                else:
                    new_col_seas_p[index] = point_dict[matchday.result]
                    new_col_seas_mp[index] = point_dict[matchday.result]

                if matchday.opponent_id not in last_year_game_res[z].keys():
                    #print("club: ", club, "\n z: ", z, "\n oppom: ", matchday.opponent_id)
                    last_year_game_res[z][matchday.opponent_id] = []
                    last_year_game_res[z][matchday.opponent_id].append(matchday.result)
                else:
                    last_year_game_res[z][matchday.opponent_id].append(matchday.result)

                if z > 0:
                    if (index > len(season)/2) & (matchday.opponent_id in last_year_game_res[z - 1].keys()):
                        new_col_last_year_game[index] = np.mean(last_year_game_res[z - 1][matchday.opponent_id] + last_year_game_res[z][matchday.opponent_id])
                    elif (index <= len(season)/2) & (matchday.opponent_id in last_year_game_res[z - 1].keys()):
                        new_col_last_year_game[index] = np.mean(last_year_game_res[z - 1][matchday.opponent_id])
                    else:
                        pass
                index += 1
            new_column_points.append(new_col_seas_p)
            new_column_meanpoints.append(new_col_seas_mp)
            #print(z, sum(new_col_last_year_game))
            last_year_game.append(new_col_last_year_game)
            z += 1
        new_column_points = list(itertools.chain.from_iterable(new_column_points))
        new_column_meanpoints = list(itertools.chain.from_iterable(new_column_meanpoints))
        last_year_game = list(itertools.chain.from_iterable(last_year_game))
        clubs[club]["goal_diff"] = goal_diff_seasons
        clubs[club]["points"] = new_column_points
        clubs[club]["mean_points"] = new_column_meanpoints
        clubs[club]["last_results"] = last_year_game
        clubs[club].loc[:,['points', 'mean_points', "last_results"]] = object.fit_transform(clubs[club].loc[:,['points', 'mean_points', "last_results"]])
    return clubs

In [ ]:
result_dict = new_data_test.return_dicts("result")
def points_and_co_oppon(clubs, result_dict):
    point_dict = {result_dict['W']: 3, result_dict['L']: 0, result_dict['D']: 1}
    for club in clubs:
        #print(clubs[club])
        oppon_points = []
        oppon_meanpoints = []
        for season in clubs[club].groupby("season_str"):
            season = season[1]
            new_col_seas_oppon_p = [0] * len(season)
            new_col_seas_oppon_mp = [0] * len(season)
            index = 0
            #print(season)
            for _, matchday in season.iterrows():
                if len(clubs[matchday.opponent_id].loc[:clubs[matchday.opponent_id][clubs[matchday.opponent_id].match_id == matchday.match_id].index[0],]) > 1:
                    new_col_seas_oppon_p[index] = clubs[matchday.opponent_id].loc[:clubs[matchday.opponent_id][clubs[matchday.opponent_id].match_id == matchday.match_id].index[0],].iloc[-2,:].points
                    new_col_seas_oppon_mp[index] = clubs[matchday.opponent_id].loc[:clubs[matchday.opponent_id][clubs[matchday.opponent_id].match_id == matchday.match_id].index[0],].iloc[-2,:].mean_points
                    if clubs[matchday.opponent_id].loc[:clubs[matchday.opponent_id][clubs[matchday.opponent_id].match_id == matchday.match_id].index[0],].iloc[-2,:].schedule_date == matchday.schedule_date:
                        print("Alarm")
                else:
                    pass

                index += 1
            oppon_points.append(new_col_seas_oppon_p)
            oppon_meanpoints.append(new_col_seas_oppon_mp)
        oppon_points = list(itertools.chain.from_iterable(oppon_points))
        oppon_meanpoints = list(itertools.chain.from_iterable(oppon_meanpoints))
        clubs[club]["oppon_points"] = oppon_points
        clubs[club]["oppon_mean_points"] = oppon_meanpoints
        
    return clubs

In [ ]:
clubs[matchday.opponent_id].loc[:clubs[matchday.opponent_id][clubs[matchday.opponent_id].match_id == matchday.match_id].index[0],].iloc[-2,:].schedule_date

In [ ]:
matchday.schedule_date

In [ ]:
def game_dict(scale_df):
    games_train = {}
    for seas in scale_df.season_str.unique()[:-1]:
        for week in sorted(scale_df.schedule_round.unique()):
            if week not in games_train.keys():
                games_train[week] = []
            df = scale_df[(scale_df.season_str == seas) & (scale_df.schedule_round == week)]
            for game in df.match_id:
                games_train[week].append(game)

    games_test = {}
    seas = scale_df.season_str.unique()[-1]
    for week in sorted(scale_df.schedule_round.unique()):
        if week not in games_test.keys():
            games_test[week] = []
        df = scale_df[(scale_df.season_str == seas) & (scale_df.schedule_round == week)]
        for game in df.match_id:
            games_test[week].append(game)
                
    return games_train, games_test

In [ ]:
game_train, game_valid = game_dict(scale_df)

In [ ]:
list(clubs[12].columns)

In [ ]:
earrange_list = [ 'result',
     'gf',
     'ga',
     'goal_diff',
     'xg',
     'xga',
     'shooting_standard_gls',
     'shooting_standard_sh',
     'shooting_standard_sot',
     'shooting_standard_sot_perc',
     'shooting_standard_g_per_sh',
     'shooting_standard_g_per_sot',
     'shooting_standard_dist',
     'shooting_standard_fk',
     'shooting_standard_pk',
     'shooting_standard_pkatt',
     'shooting_expected_npxg',
     'shooting_expected_npxg_per_sh',
     'shooting_expected_g_minus_xg',
     'shooting_expected_npg_minus_xg',
     'keeper_performance_sota',
     'keeper_performance_saves',
     'keeper_performance_save_perc',
     'keeper_performance_cs',
     'keeper_performance_psxg',
     'keeper_performance_psxg_plus_minus',
     'keeper_penaltykicks_pkatt',
     'keeper_penaltykicks_pka',
     'keeper_penaltykicks_pksv',
     'keeper_penaltykicks_pkm',
     'keeper_launched_cmp',
     'keeper_launched_att',
     'keeper_launched_cmp_perc',
     'keeper_passes_att',
     'keeper_passes_thr',
     'keeper_passes_launch_perc',
     'keeper_passes_avglen',
     'keeper_goalkicks_att',
     'keeper_goalkicks_launch_perc',
     'keeper_goalkicks_avglen',
     'keeper_crosses_opp',
     'keeper_crosses_stp',
     'keeper_crosses_stp_perc',
     'keeper_sweeper_number_opa',
     'keeper_sweeper_avgdist',
     'passing_total_cmp',
     'passing_total_att',
     'passing_total_cmp_perc',
     'passing_total_totdist',
     'passing_total_prgdist',
     'passing_short_cmp',
     'passing_short_att',
     'passing_short_cmp_perc',
     'passing_medium_cmp',
     'passing_medium_att',
     'passing_medium_cmp_perc',
     'passing_long_cmp',
     'passing_long_att',
     'passing_long_cmp_perc',
     'passing_attacking_ast',
     'passing_attacking_xag',
     'passing_attacking_xa',
     'passing_attacking_kp',
     'passing_attacking_1_per_3',
     'passing_attacking_ppa',
     'passing_attacking_crspa',
     'passing_attacking_prgp',
     'passing_types_passtypes_live',
     'passing_types_passtypes_dead',
     'passing_types_passtypes_fk',
     'passing_types_passtypes_tb',
     'passing_types_passtypes_sw',
     'passing_types_passtypes_crs',
     'passing_types_passtypes_ti',
     'passing_types_passtypes_ck',
     'passing_types_cornerkicks_in',
     'passing_types_cornerkicks_out',
     'passing_types_cornerkicks_str',
     'passing_types_outcomes_off',
     'passing_types_outcomes_blocks',
     'gca_scatypes_sca',
     'gca_scatypes_passlive',
     'gca_scatypes_passdead',
     'gca_scatypes_to',
     'gca_scatypes_sh',
     'gca_scatypes_fld',
     'gca_scatypes_def',
     'gca_gcatypes_gca',
     'gca_gcatypes_passlive',
     'gca_gcatypes_passdead',
     'gca_gcatypes_to',
     'gca_gcatypes_sh',
     'gca_gcatypes_fld',
     'gca_gcatypes_def',
     'defense_tackles_tkl',
     'defense_tackles_tklw',
     'defense_tackles_def3rd',
     'defense_tackles_mid3rd',
     'defense_tackles_att3rd',
     'defense_challenges_tkl',
     'defense_challenges_att',
     'defense_challenges_tkl_perc',
     'defense_challenges_lost',
     'defense_blocks_blocks',
     'defense_blocks_sh',
     'defense_blocks_pass',
     'defense_general_int',
     'defense_general_tkl_plus_int',
     'defense_general_clr',
     'defense_general_err',
     'possession_general_poss',
     'possession_touches_touches',
     'possession_touches_defpen',
     'possession_touches_def3rd',
     'possession_touches_mid3rd',
     'possession_touches_att3rd',
     'possession_touches_attpen',
     'possession_touches_live',
     'possession_takeons_att',
     'possession_takeons_succ',
     'possession_takeons_succ_perc',
     'possession_takeons_tkld',
     'possession_takeons_tkld_perc',
     'possession_carries_carries',
     'possession_carries_totdist',
     'possession_carries_prgdist',
     'possession_carries_prgc',
     'possession_carries_1_per_3',
     'possession_carries_cpa',
     'possession_carries_mis',
     'possession_carries_dis',
     'possession_receiving_rec',
     'possession_receiving_prgr',
     'misc_performance_crdy',
     'misc_performance_crdr',
     'misc_performance_2crdy',
     'misc_performance_fls',
     'misc_performance_fld',
     'misc_performance_off',
     'misc_performance_og',
     'misc_performance_recov',
     'misc_aerialduels_won',
     'misc_aerialduels_lost',
     'misc_aerialduels_won_perc',
     'attendance',
     'points',
     'mean_points',
     'weekly_wages_eur',
     'season_str',
     'league_id',
     'venue',
     'team_id',
     'opponent_id',
     "last_results",
     'oppon_points',
     'oppon_mean_points',
     'schedule_round',
     'formation',
     'captain',
     'referee',
     'schedule_day',
     'match_id',
     'schedule_date',
     'schedule_time',
     ]
def inputs(games, clubs, rearrange_list):

    lstm_inputs = [[], [], [], [], []] # [[input1], [input2], [result1], [gf1], [ga1], [g_diff1], [result2], [gf2], [ga2], [g_diff2], [features_nxt_game1], [features_nxt_game2]]
    for i in range(7,len(games)): # not plus 1 as last matchday should yield no result, investigate!
        #lstm_inputs[i] = [[], [], []]
        print(i)
        for game in games[i]:
                # get ids of participating teams
            team1 = scale_df[scale_df.match_id == game].iloc[0].team_id
            team2 = scale_df[scale_df.match_id == game].iloc[1].team_id

            df_team1 = clubs[team1]
            df_team1 = df_team1[rearrange_list]

            season = df_team1[df_team1.match_id == game].season_str.values[0]  # to only take history of current season into account
            df_team1_past = df_team1.loc[df_team1[df_team1.season_str == season].iloc[0].name:df_team1[df_team1.match_id == game].index.values[0],:]  # df with all predecessing games in this season

            df_team2 = clubs[team2]#  get teams df
            df_team2 = df_team2[rearrange_list]

            df_team2_past = df_team2.loc[df_team2[df_team2.season_str == season].iloc[0].name:df_team2[df_team2.match_id == game].index.values[0],:]  # df with all predecessing games in this season

                # create zeros np array to store date
            np_team1 = np.zeros([len(games), df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
            np_team2 = np.zeros([len(games), df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])

         #   print("np_empty", np_team1.shape)
          #  print("np", np_team1[- len(df_team1_past):, :-df_team1_past.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]].shape)
           # print("df", df_team1_past.loc[:, "weekly_wages_eur":"schedule_round"].shape)

                # insert data into dataframe (back to front) such that all input into the lstm has the same length (padding)
            np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
            np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]

                 # tensor for pytorch lstm 
                #np_team1 = torch.from_numpy(np_team1)
                #np_team2 = torch.from_numpy(np_team2)

                # if last game, no do not add as no future results to predict
            if df_team1.iloc[-1].name == df_team1_past.iloc[-1].name:
                pass

            elif df_team2.iloc[-1].name == df_team2_past.iloc[-1].name:
                pass

                # if not last game, add to input dict together with result
            else:
                np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"weekly_wages_eur":"schedule_round"]  # df with all predecessing games in this season
                np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"weekly_wages_eur":"schedule_round"]  # df with all predecessing games in this season
                
                results1 = np.zeros((len(games), 4))
                results2 = np.zeros((len(games), 4))
                #print(result1)
                res1 = len(df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                res2 = len(df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                #print(result1)
                result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
                result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
                lstm_inputs[0].append(torch.tensor(np_team1))
                lstm_inputs[1].append(torch.tensor(np_team2))
                lstm_inputs[2].append(result1)
                lstm_inputs[3].append(result2)
                #lstm_inputs[4].append(fut_feature1)
                #lstm_inputs[5].append(fut_feature2)
    return lstm_inputs

In [ ]:
train_inputs = inputs(game_train, clubs)
valid_inputs = inputs(game_valid, clubs)

In [ ]:
class data_to_lstm():

    def __init__(self, mylist):
        self.output = []
        self.team_home = mylist[0]
        self.team_away = mylist[1]
        self.result1 = mylist[2]
        self.result2 = mylist[3]
        #self.team_home_next = mylist[4]
        #self.team_away_next = mylist[5]
        
        
        for i in range(len(self.team_home)):
            self.output.append(((self.team_home[i], self.team_away[i]), (self.result1[i], self.result2[i])))#, (self.team_home_next[i], self.team_away_next[i])))
        

    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.output[idx]

In [ ]:
train_for_loader = data_to_lstm(train_inputs)
valid_for_loader = data_to_lstm(valid_inputs)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_for_loader, batch_size = 32, drop_last = True)
test_loader = torch.utils.data.DataLoader(valid_for_loader, batch_size = 32, drop_last = True)

In [ ]:
class Sport_pred_2LSTM_1(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes, num_layers = 1):
        super(Sport_pred_2LSTM_1, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = num_layers # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(self.n_hidden, self.num_classes)
        #self.l_linear2 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.ReLU()
        self.sigm = torch.nn.Sigmoid()


    def forward(self, x):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        ht1 = ht1.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        #print(lstm_out1.shape)
        out = lstm_out1#[:,-1,:]#.contiguous().view(-1, self.n_hidden)
        #print(out.shape)
        
        # run lstm output through nn to predict outcome
        result = self.l_linear1(out)
        #result = self.l_linear2(result)
        #result = self.soft(result)
        #result = self.sigm(result)
        return result

In [ ]:
class Sport_pred_2LSTM_2(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM_2, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        #self.relu = torch.nn.SiLU()


    def forward(self, x):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        
        out = lstm_out1#.contiguous().view(-1, self.n_hidden)

        result = self.l_linear1(out)
        #result = self.soft(result)
        return result

In [29]:
def train_sport_pred_2LSTM(lstm1, lstm2, train_loader, test_loader, learn_rate, epochs):
    criterion1 = torch.nn.MSELoss()
    criterion2 = torch.nn.MSELoss()
    optimizer1 = torch.optim.SGD(lstm1.parameters(), lr = learn_rate, weight_decay = 0.01)
    optimizer2 = torch.optim.SGD(lstm2.parameters(), lr = learn_rate, weight_decay = 0.01)
    
    lstm1.eval()
    lstm2.eval()
    accur1 = 0
    accur2 = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader): #, (next_feat1, next_feat2)
        pred1 = lstm1(input1)
        pred2 = lstm2(input2)
        #print(pred.shape)
        pred1 = torch.argmax(pred1, dim = 1)
        pred2 = torch.argmax(pred2, dim = 1)
        #print(pred.shape)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3)
        result1 = torch.argmax(result1, dim = 1)
        result2 = torch.argmax(result2, dim = 1)
        accur1 += pred1.eq(result1).sum().item()
        accur2 += pred2.eq(result2).sum().item()
    print(f"Loss accuracy training 1: {100 * accur1 /((step + 1) * 32)}% \n Loss accuracy training 2: {100 * accur2 /((step + 1) * 32)}%")
    
    for epoch in range(epochs):
        losses_val1 = []
        losses_val2 = []
        accuracies1 = []
        accuracies2 = []
        
        lstm1.train()
        lstm2.train()
        print(f"Epoch: {epoch}")
        for step, ((input1, input2), (result1, result2)) in enumerate(train_loader): #, (next_feat1, next_feat2)
            lstm1.zero_grad()
            lstm2.zero_grad()
            pred1 = lstm1(input1)
            pred2 = lstm2(input2)
            #result1 = torch.nn.functional.one_hot(result1.to(torch.int64), num_classes = 3).to(torch.float32)
            #result2 = torch.nn.functional.one_hot(result2.to(torch.int64), num_classes = 3).to(torch.float32)
            #print(f"pred {pred.dtype}, result {result.dtype}")
            loss1 = criterion1(pred1.to(torch.float32), result1.to(torch.float32))
            loss2 = criterion2(pred2.to(torch.float32), result2.to(torch.float32))
            loss1.backward()
            loss2.backward()
            optimizer1.step()
            optimizer2.step()
        
        lstm1.eval()
        lstm2.eval()
        loss1 = 0
        loss2 = 0
        accur1 = 0
        accur2 = 0
        for step, ((input1, input2), (result1, result2)) in enumerate(test_loader): #, (next_feat1, next_feat2)
            pred1 = lstm1(input1)
            pred2 = lstm1(input2)
            #result_oh1 = result1.to(torch.int64)
            #result_oh2 = result2.to(torch.int64)
            #print(result.shape)
            #result_oh1 = torch.nn.functional.one_hot(result_oh1, num_classes = 3)
            #result_oh2 = torch.nn.functional.one_hot(result_oh2, num_classes = 3)
            #print(result.shape)
            #print(pred.shape)
            loss1 += criterion1(pred1, result1.to(torch.float32)).item()
            loss2 += criterion2(pred2, result2.to(torch.float32)).item()
            result1 = torch.argmax(result1, dim = 1)[-1]
            result2 = torch.argmax(result2, dim = 1)[-1]
            pred1 = torch.argmax(pred1, dim = 1)[-1]
            pred2 = torch.argmax(pred2, dim = 1)[-1]
            #print(pred.shape)
            #print(result1.shape)
            accur1 += pred1.eq(result1).sum().item()
            accur2 += pred2.eq(result2).sum().item()
        losses_val1.append(loss1)
        losses_val2.append(loss2)
        accuracy1 = 100 * (accur1 /((step + 1) * 32))
        accuracy2 = 100 * (accur2 /((step + 1) * 32))
        accuracies1.append(accuracy1)
        accuracies2.append(accuracy2)
        print(f"Loss 1 {loss1} accuracy 1 {accuracy1}% \n Loss 2 {loss2} accuracy 2 {accuracy2}% ") 
                
        

In [ ]:

#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(153, 153, 3, 2)
print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 1e-5, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    #print(pred.shape)
    #print(result1.shape)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    accur += pred.eq(result).sum().item()

    pred = net(input2)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result2 = result2[:,-1,:]
    result2 = torch.argmax(result2, dim = 1)
    accur += pred.eq(result2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 64)}%")

for epoch in range(25):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        #result1 = result1[:,-1,:]
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        #result2 = result2[:,-1,:]
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1)
        #print(pred)
        #print(pred.shape)
        result1 = result1[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result1).item()
        #
        #print(pred)
        #print(pred.shape)
        pred = torch.argmax(pred, dim = 1)
        #print(pred)
        #print(pred.shape)

        result1 = torch.argmax(result1, dim = 1)
        accur += pred.eq(result1).sum().item()
       # if epoch == 9:
        #    print("pred", pred)
         #   print("result", result1)
        pred = net(input2)
        result2 = result2[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result2).item()
        #pred = pred[:,-1,:]
        pred = torch.argmax(pred, dim = 1)
        result2  = torch.argmax(result2, dim = 1)
        accur += pred.eq(result2).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 64))
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


In [ ]:
test_loader

In [ ]:
img = next(iter(test_loader))

In [ ]:
img

In [ ]:
class res_to_res(torch.nn.Module):
    def __init__(self, n_features, num_layers):
        super(res_to_res, self).__init__()
        self.n_features = n_features 
        self.num_layers = num_layers
        
        self.fcs = []
        self.activation = torch.nn.Sigmoid()
        self.soft = torch.nn.Softmax()
        
        for i in range(self.num_layers):               # build hidden layers 
            input_size = n_features if i == 0 else 10
            fc = torch.nn.Linear(input_size, 10)
            setattr(self, 'fc%i' % i, fc)       # set layer to the Module
            self._set_init(fc)                  # parameters initialization
            self.fcs.append(fc)

        self.predict = torch.nn.Linear(10, 3)         # output layer
        self._set_init(self.predict)            # parameters initialization

    def _set_init(self, layer):
        torch.nn.init.normal_(layer.weight, mean=0., std=.1)
        #torch.nn.init.constant_(layer.bias, self.bias_init)

    def forward(self, x):        
        for i in range(self.num_layers):
            x = self.fcs[i](x)
            x = self.activation(x)
        
        out = self.predict(x)
        return out

In [ ]:
net2 = res_to_res(6,3)

#net = Sport_pred_2LSTM_1(151, 151, 3)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net2.parameters(), lr = 1e-3)
    
net.eval()
net2.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)
    pred2 = net(input2)
    pred = torch.concat([pred1, pred2], dim = 2)[:,-1,:]
    pred = net2(pred)
    result = result1[:,-1,:]
    pred = torch.argmax(pred, dim = 1)
    result = torch.argmax(result, dim = 1)
    accur += pred.eq(result).sum().item()
    
print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

for epoch in range(10):
    losses_val = []
    accuracies = []

    net2.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net2.zero_grad()
        pred1 = net(input1)
        pred2 = net(input2)
        pred = torch.concat([pred1, pred2], dim = 2)[:,-1,:]
        pred = net2(pred)
        result1 = result1[:,-1,:]
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()


    net2.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
            pred1 = net(input1)
            pred2 = net(input2)
            pred = torch.concat([pred1, pred2], dim = 2)[:,-1,:]
            pred = net2(pred)
            result = result1[:,-1,:]
            loss += criterion(pred, result).item()
            pred = torch.argmax(pred, dim = 1)
            result = torch.argmax(result, dim = 1)
            accur += pred.eq(result).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 32))
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 



In [ ]:
pred.shape

In [ ]:
torch.concat([pred1, pred2], dim = 2).shape

In [46]:
class Sport_pred_2LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.ReLU()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x,(h02, c02))
        ht1 = ht1.squeeze()#[-1,:]
        ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1[:,-1,:]
        y = lstm_out2[:,-1,:]
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        #in_lay1 = torch.cat((ht1, ct1, ht2, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.l_linear4(in_lay4)
        #print("out", out.shape)
        
        #print(out)
        return out

In [42]:
next(iter(test_loader))[0][0].shape[2]


366

In [47]:
len_input = next(iter(test_loader))[0][0].shape[2]
net = Sport_pred_2LSTM(len_input, len_input, 3)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 1e-5, weight_decay = 0.1)
    
net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    #print(input1.shape)
    pred = net(input1, input2)
    pred = torch.argmax(pred, dim = 1)
    result1 = result1[:,-1,:]
    result1 = torch.argmax(result1, dim = 1)
    accur += pred.eq(result1).sum().item()

print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

for epoch in range(10):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1, input2)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        result1 = result1[:,-1,:]
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()


    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1, input2)
        #print(pred)
        #print(pred.shape)
        result1 = result1[:,-1,:]
        loss += criterion(pred, result1).item()
        pred = torch.argmax(pred, dim = 1)
        #print(pred)
        #print(pred.shape)
        result1 = torch.argmax(result1, dim = 1)
        accur += pred.eq(result1).sum().item()
        
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 63))
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 



Loss accuracy training: 38.72282608695652%
Epoch: 0
Loss 21.013176426291466 accuracy 19.66873706004141%
Epoch: 1
Loss 20.596253916621208 accuracy 19.66873706004141%
Epoch: 2
Loss 20.46710443496704 accuracy 19.66873706004141%
Epoch: 3
Loss 20.405873775482178 accuracy 19.66873706004141%
Epoch: 4
Loss 20.362672597169876 accuracy 19.66873706004141%
Epoch: 5


KeyboardInterrupt: 

In [ ]:
print(result1[:,-1,:].shape)

In [ ]:
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1)
        loss += criterion(pred, result1).item()
        print(pred.shape)
        print(pred)
        pred = pred[:,-1,:]
        pred = torch.argmax(pred, dim = 1)
        print(pred.shape)
        print(pred)
        result1 = result1[:,-1,:]
        result1 = torch.argmax(result1, dim = 1)
        accur += pred.eq(result1).sum().item()
        print("pred", pred)
        print("result", result1)
        break

In [ ]:
pred

In [ ]:
pred.eq(result2)#.sum().item()#/64

In [ ]:
result2.eq(2)

In [ ]:
new_data_test.return_dicts("result")

In [ ]:
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    print(result1[-1,:,:].shape)
    print(result1.shape)

In [ ]:
new_data = preprocess(data)
new_data1 = new_data.data_frame#.dataset_team("e0652b02").captain.unique()

In [ ]:
new_data.data_frame

In [ ]:
scale_df = new_data.data_frame

liste = ['schedule_time', 'schedule_round', 'schedule_day', 'result_home', 'gf_home', 'gf_away', 'xg_home',
         'xg_away', 'formation_home', 'referee', 'fbref_season', 'fbref_league_id', 'fbref_home_id',
         'fbref_away_id', 'fbref_match_id', 'result_away', 'captain_away', 'formation_away', 'captain_home',
         'captain_away']

cols_to_scale = list(set(list(scale_df.columns)).difference(liste))
object = StandardScaler()

scale_df.loc[:,cols_to_scale] = object.fit_transform(scale_df.loc[:,cols_to_scale])
scale_df.head()

In [ ]:
def game_dict(scale_df):
    games_train = {}
    for seas in scale_df.fbref_season.unique()[:-1]:
        for week in sorted(scale_df.schedule_round.unique()):
            if week not in games_train.keys():
                games_train[week] = []
            df = scale_df[(scale_df.fbref_season == seas) & (scale_df.schedule_round == week)]
            for game in df.fbref_match_id:
                games_train[week].append(game)

    games_test = {}
    seas = scale_df.fbref_season.unique()[-1]
    for week in sorted(scale_df.schedule_round.unique()):
        if week not in games_test.keys():
            games_test[week] = []
        df = scale_df[(scale_df.fbref_season == seas) & (scale_df.schedule_round == week)]
        for game in df.fbref_match_id:
            games_test[week].append(game)
                
    return games_train, games_test
#games

In [ ]:
game_train, game_valid = game_dict(scale_df)

In [ ]:
def concat_preprocess(df, where):
        colname_dict = {}
        result_dict = new_data.return_dicts("result")
        for colname in df.loc[:, df.columns.str.endswith(where)]:
            colname_dict[colname]  = colname[:-5]
    
        return df.rename(colname_dict, axis = 1, inplace = True)

In [ ]:
def dataset_team(data_frame, team_id):
        # split dataset 
        random_team1 = data_frame[data_frame.fbref_home_id == team_id].loc[:, data_frame.columns.str.endswith('home')]
        random_team2 = data_frame[data_frame.fbref_away_id == team_id].loc[:, data_frame.columns.str.endswith('away')]
        random_team3 = data_frame[(data_frame.fbref_home_id == team_id) |
                        (data_frame.fbref_away_id == team_id)].loc[:,['schedule_time', 'attendance',
                                                                           'referee', 'fbref_season',
                                                                           'fbref_league_id', 'fbref_home_id',
                                                                           'fbref_away_id', 'fbref_match_id']]

        #random_team2.rename({"gf_away": "gf_home", "gf_home": "gf_away"}, axis = 1, inplace = True)
        # preprocess chunks
        concat_preprocess(random_team1, "home")
        concat_preprocess(random_team2, "away")
        random_team = pd.concat([random_team1, random_team2])

        # dummy to account for home and away
        random_team3["home"] = np.where(random_team3.fbref_home_id == team_id, 1, 0)

        # rename columns to change them
        random_team3_away = random_team3.loc[random_team3.fbref_away_id == team_id].copy()
        random_team3_away.rename({"fbref_home_id": "fbref_away_id",
                                  "fbref_away_id": "fbref_home_id"}, axis = 1, inplace = True)
        random_team3_home = random_team3[random_team3.fbref_home_id == team_id]

        # concat after changing column names and rename 
        random_team3 = pd.concat([random_team3_home, random_team3_away])
        random_team3.rename({"fbref_home_id": "fbref_own", 
                             "fbref_away_id": "fbref_oppon",
                             "gf_home": "gf_own", 
                             "gf_away": "gf_own"}, axis = 1, inplace = True)

        end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()

        return end_team

In [ ]:
clubs = {}
for club in set(list(scale_df.fbref_home_id) + list(scale_df.fbref_away_id)):
    clubs[club] = dataset_team(scale_df, club)

In [ ]:
def inputs(games, clubs):
    #lstm_inputs = {}

    rearrange_list = ['result', 'gf', 'xg', 'captain', 'formation',
     'shooting_standard_gls', 'shooting_standard_sh', 'shooting_standard_sot', 'shooting_standard_sot_perc',
     'shooting_standard_g_per_sh', 'shooting_standard_g_per_sot', 'shooting_standard_dist', 'shooting_standard_fk',
     'shooting_standard_pk', 'shooting_standard_pkatt', 'shooting_expected_npxg', 'shooting_expected_npxg_per_sh',
     'shooting_expected_g_minus_xg', 'shooting_expected_npg_minus_xg', 'keeper_performance_sota', 'keeper_performance_saves',
     'keeper_performance_save_perc', 'keeper_performance_cs', 'keeper_performance_psxg', 'keeper_performance_psxg_plus_minus',
     'keeper_penaltykicks_pkatt', 'keeper_penaltykicks_pksv', 'keeper_penaltykicks_pkm', 'keeper_launched_cmp',
     'keeper_launched_att', 'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr', 'keeper_passes_launch_perc',
     'keeper_passes_avglen', 'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc', 'keeper_goalkicks_avglen',
     'keeper_crosses_opp', 'keeper_crosses_stp', 'keeper_crosses_stp_perc', 'keeper_sweeper_number_opa', 'keeper_sweeper_avgdist',
     'passing_total_cmp', 'passing_total_att', 'passing_total_cmp_perc', 'passing_total_totdist', 'passing_total_prgdist',
     'passing_short_cmp', 'passing_short_att', 'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
     'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att', 'passing_long_cmp_perc', 'passing_attacking_ast',
     'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp', 'passing_attacking_1_per_3',
     'passing_attacking_ppa', 'passing_attacking_crspa', 'passing_attacking_prgp', 'passing_types_passtypes_live',
     'passing_types_passtypes_dead', 'passing_types_passtypes_fk', 'passing_types_passtypes_tb', 'passing_types_passtypes_sw',
     'passing_types_passtypes_crs', 'passing_types_passtypes_ti', 'passing_types_passtypes_ck', 'passing_types_cornerkicks_in',
     'passing_types_cornerkicks_out', 'passing_types_cornerkicks_str', 'passing_types_outcomes_off', 'passing_types_outcomes_blocks',
     'gca_scatypes_sca', 'gca_scatypes_passlive', 'gca_scatypes_passdead', 'gca_scatypes_to', 'gca_scatypes_sh',
     'gca_scatypes_fld', 'gca_scatypes_def', 'gca_gcatypes_gca', 'gca_gcatypes_passlive','gca_gcatypes_passdead',
     'gca_gcatypes_to', 'gca_gcatypes_sh', 'gca_gcatypes_fld', 'gca_gcatypes_def', 'defense_tackles_tkl', 'defense_tackles_tklw',
     'defense_tackles_def3rd', 'defense_tackles_mid3rd', 'defense_tackles_att3rd', 'defense_challenges_tkl',
     'defense_challenges_att', 'defense_challenges_tkl_perc', 'defense_challenges_lost', 'defense_blocks_blocks',
     'defense_blocks_sh', 'defense_blocks_pass', 'defense_general_int', 'defense_general_tkl_plus_int',
     'defense_general_clr', 'defense_general_err', 'possession_general_poss', 'possession_touches_touches',
     'possession_touches_defpen', 'possession_touches_def3rd', 'possession_touches_mid3rd', 'possession_touches_att3rd',
     'possession_touches_attpen', 'possession_touches_live', 'possession_takeons_att', 'possession_takeons_succ',
     'possession_takeons_succ_perc', 'possession_takeons_tkld', 'possession_takeons_tkld_perc', 'possession_carries_carries', 
     'possession_carries_totdist', 'possession_carries_prgdist', 'possession_carries_prgc', 'possession_carries_1_per_3',
     'possession_carries_cpa', 'possession_carries_mis', 'possession_carries_dis', 'possession_receiving_rec', 
     'possession_receiving_prgr', 'misc_performance_crdy', 'misc_performance_crdr', 'misc_performance_2crdy',
     'misc_performance_fls', 'misc_performance_fld', 'misc_performance_off', 'misc_performance_og', 'misc_performance_recov', 
     'misc_aerialduels_won', 'misc_aerialduels_lost', 'misc_aerialduels_won_perc', 'keeper_penaltykicks_pka', 'attendance',
     'schedule_time', 'referee', 'fbref_season', 'fbref_league_id', 'fbref_own', 'fbref_oppon', 'fbref_match_id', 'home']
    lstm_inputs = [[], [], [], [], []]
    for i in range(7,len(games)): # not plus 1 as last matchday should yield no result, investigate!
        #lstm_inputs[i] = [[], [], []]
        print(i)
        for game in games[i]:
            # get ids of participating teams
            team1 = int(scale_df[scale_df.fbref_match_id == game].fbref_home_id.values)
            team2 = int(scale_df[scale_df.fbref_match_id == game].fbref_away_id.values)

            #  get teams df
            df_team1 = clubs[team1]#  get teams df
            df_team1 = df_team1[rearrange_list]

            season = df_team1[df_team1.fbref_match_id == game].fbref_season.values[0]  # to only take history of current season into account
            df_team1_past = df_team1.loc[df_team1[df_team1.fbref_season == season].iloc[0].name:df_team1[df_team1.fbref_match_id == game].index.values[0],:]  # df with all predecessing games in this season

            df_team2 = clubs[team2]#  get teams df
            df_team2 = df_team2[rearrange_list]

            df_team2_past = df_team2.loc[df_team2[df_team2.fbref_season == season].iloc[0].name:df_team2[df_team2.fbref_match_id == game].index.values[0],:]  # df with all predecessing games in this season

            # create zeros np array to store date
            np_team1 = np.zeros([len(games), df_team1_past.loc[:,"xg":].shape[1]])
            np_team2 = np.zeros([len(games), df_team2_past.loc[:,"xg":].shape[1]])

            # insert data into dataframe (back to front) such that all input into the lstm has the same length (padding)
            np_team1[- len(df_team1_past):, :-8] = df_team1_past.loc[:,"xg": "attendance"]
            np_team2[- len(df_team2_past):, :-8] = df_team2_past.loc[:,"xg": "attendance"]

             # tensor for pytorch lstm 
            #np_team1 = torch.from_numpy(np_team1)
            #np_team2 = torch.from_numpy(np_team2)

            # if last game, no do not add as no future results to predict
            if df_team1.iloc[-1].name == df_team1_past.iloc[-1].name:
                pass

            elif df_team2.iloc[-1].name == df_team2_past.iloc[-1].name:
                pass

            # if not last game, add to input dict together with result
            else:
                np_team1[- len(df_team1_past):, -8:] = df_team1.loc[df_team1[df_team1.fbref_season == season].iloc[1].name:df_team1.loc[df_team1[df_team1.fbref_match_id == game].index.values[0]:,:].iloc[1].name,"schedule_time":]  # df with all predecessing games in this season
                np_team2[- len(df_team2_past):, -8:] = df_team2.loc[df_team2[df_team2.fbref_season == season].iloc[1].name:df_team2.loc[df_team2[df_team2.fbref_match_id == game].index.values[0]:,:].iloc[1].name,"schedule_time":]  # df with all predecessing games in this season

                result = df_team1.loc[df_team1[df_team1.fbref_match_id == game].index.values[0]:,:].iloc[1].result
                fut_feature1 = df_team1.loc[df_team1[df_team1.fbref_match_id == game].index.values[0]:,:].iloc[1,2:].values
                fut_feature2 = df_team2.loc[df_team2[df_team2.fbref_match_id == game].index.values[0]:,:].iloc[1,2:].values

                lstm_inputs[0].append(np_team1)
                lstm_inputs[1].append(np_team2)
                lstm_inputs[2].append(result)
                lstm_inputs[3].append(fut_feature1)
                lstm_inputs[4].append(fut_feature2)
    return lstm_inputs

In [ ]:
train_inputs = inputs(game_train, clubs)
valid_inputs = inputs(game_valid, clubs)

In [ ]:
class data_to_lstm():

    def __init__(self, mylist):
        self.output = []
        self.team_home = mylist[0]
        self.team_away = mylist[1]
        self.result = mylist[2]
        self.team_home_next = mylist[3]
        self.team_away_next = mylist[4]
        
        
        for i in range(len(self.team_home)):
            self.output.append(((self.team_home[i], self.team_away[i]),self.result[i], (self.team_home_next[i], self.team_away_next[i])))
        

    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.output[idx]


In [ ]:
train_for_loader = data_to_lstm(train_inputs)
valid_for_loader = data_to_lstm(valid_inputs)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_for_loader, batch_size = 32, drop_last = True)
test_loader = torch.utils.data.DataLoader(valid_for_loader, batch_size = 32, drop_last = True)

In [ ]:
class Sport_pred_2LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(4 * self.n_hidden, 4 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(4 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.SiLU()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x)#,(h02, c02))
        ht1 = ht1.squeeze()#[-1,:]
        ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1.contiguous().view(-1, self.n_hidden)
        y = lstm_out2.contiguous().view(-1, self.n_hidden)
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        #in_lay1 = torch.cat((x, y), 1)
        in_lay1 = torch.cat((ht1, ct1, ht2, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.soft(self.l_linear4(in_lay4))
        #print("out", out.shape)
        
        #print(out)
        return out

In [ ]:
def train_sport_pred_2LSTM(lstm, train_loader, test_loader, learn_rate, epochs):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(lstm.parameters(), lr = learn_rate, weight_decay = 0.2)
    
    lstm.eval()
    accur = 0
    for step, ((input1, input2), result, (next_feat1, next_feat2)) in enumerate(test_loader):
        pred = lstm(input1, input2)
        #print(pred.shape)
        pred = torch.argmax(pred, dim = 1)
        #print(pred.shape)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3)
        accur += pred.eq(result).sum().item()
    print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")
    
    for epoch in range(epochs):
        losses_val = []
        accuracies = []
        
        lstm.train()
        print(f"Epoch: {epoch}")
        for step, ((input1, input2), result, (next_feat1, next_feat2)) in enumerate(train_loader):
            lstm.zero_grad()
            pred = lstm(input1, input2)
            result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
            #print(f"pred {pred.dtype}, result {result.dtype}")
            loss = criterion(pred, result)
            loss.backward()
            optimizer.step()
        
        lstm.eval()
        loss = 0
        accur = 0
        for step, ((input1, input2), result, (next_feat1, next_feat2)) in enumerate(test_loader):
            pred = lstm(input1, input2)
            result_oh = result.to(torch.int64)
            #print(result.shape)
            result_oh = torch.nn.functional.one_hot(result_oh, num_classes = 3)
            #print(result.shape)
            #print(pred.shape)
            loss += criterion(pred, result_oh).item()
            pred = torch.argmax(pred, dim = 1)
            #print(pred.shape)
            #print(pred.eq(result).sum())
            accur += pred.eq(result).sum().item()
        losses_val.append(loss)
        accuracy = 100 * (accur /((step + 1) * 32))
        accuracies.append(accuracy)
        print(f"Loss {loss} accuracy {accuracy}%") 
                
        

In [ ]:
lstm1 = Sport_pred_2LSTM(150, 150, 3)
train_sport_pred_2LSTM(lstm1, train_loader, test_loader, 5e-5, 50)

In [ ]:
class Sport_pred_2LSTM_var1(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM_var1, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.ReLU()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x)#,(h02, c02))
        #ht1 = ht1.squeeze()#[-1,:]
        #ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1.contiguous().view(-1, self.n_hidden)
        y = lstm_out2.contiguous().view(-1, self.n_hidden)
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        #in_lay1 = torch.cat((x, y), 1)
        in_lay1 = torch.cat((ct1, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.soft(self.l_linear4(in_lay4))
        #print("out", out.shape)
        
        #print(out)
        return out.squeeze()

In [ ]:
lstm2 = Sport_pred_2LSTM_var1(150, 150, 3)
train_sport_pred_2LSTM(lstm2, train_loader, test_loader, 1e-4, 15)

In [ ]:
class Sport_pred_2LSTM_var2(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM_var2, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.Tanh()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x)#,(h02, c02))
        #ht1 = ht1.squeeze()#[-1,:]
        #ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1.contiguous().view(-1, self.n_hidden)
        y = lstm_out2.contiguous().view(-1, self.n_hidden)
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        #in_lay1 = torch.cat((x, y), 1)
        in_lay1 = torch.cat((ct1, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.soft(self.l_linear4(in_lay4))
        #print("out", out.shape)
        
        #print(out)
        return out.squeeze()

In [ ]:
lstm3 = Sport_pred_2LSTM_var2(150, 150, 3)
train_sport_pred_2LSTM(lstm3, train_loader, test_loader, 1e-4, 15)

In [ ]:
class Sport_pred_2LSTM_var3(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM_var3, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.Tanh()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x)#,(h02, c02))
        #ht1 = ht1.squeeze()#[-1,:]
        #ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1[:,-1,:]
        y = lstm_out2[:,-1,:]
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        #in_lay1 = torch.cat((ct1, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.soft(self.l_linear4(in_lay4))
        #print("out", out.shape)
        
        #print(out)
        return out.squeeze()

In [ ]:
lstm4 = Sport_pred_2LSTM_var3(150, 150, 3)
train_sport_pred_2LSTM(lstm4, train_loader, test_loader, 1e-3, 15)

In [ ]:
class Sport_pred_2LSTM_var4(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM_var4, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.Tanh()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x)#,(h02, c02))
        #ht1 = ht1.squeeze()#[-1,:]
        #ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1[:,-1,:]
        y = lstm_out2[:,-1,:]
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        #in_lay1 = torch.cat((ct1, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.soft(self.l_linear4(in_lay4))
        #print("out", out.shape)
        
        #print(out)
        return out.squeeze()

In [ ]:
lstm5 = Sport_pred_2LSTM_var4(150, 300, 3)
train_sport_pred_2LSTM(lstm5, train_loader, test_loader, 1e-3, 15)

In [ ]:
class Sport_pred_2LSTM_var4(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_2LSTM_var4, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear2 = torch.nn.Linear(2 * self.n_hidden, 2 * self.n_hidden)
        self.l_linear3 = torch.nn.Linear(2 * self.n_hidden, self.n_hidden)
        self.l_linear4 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.Tanh()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #h02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c02 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        lstm_out2,(ht2, ct2) = self.l_lstm2(x)#,(h02, c02))
        #ht1 = ht1.squeeze()#[-1,:]
        #ht2 = ht2.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        ct2 = ct2.squeeze()#[-1,:]
        
        #print("ht1", ht1.shape)
        #print("ht2", ht2.shape)
        #print("ct1", ct1.shape)
        #print("ct2", ct2.shape)
        #print("lstm_out1",lstm_out1.shape)
        #print("lstm_out2",lstm_out2.shape)
        
        x = lstm_out1[:,-1,:]
        y = lstm_out2[:,-1,:]
        
        #print("x", x.shape)
        #print("y", y.shape)
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        #in_lay1 = torch.cat((ct1, ct2), -1)
        #print("in_lay1", in_lay1.shape)
        out_lay1 = self.l_linear1(in_lay1)
        #print("out_lay1", out_lay1.shape)
        
        in_lay2 = self.relu(out_lay1)
        #print("in_lay2", in_lay2.shape)
        out_lay2 = self.l_linear2(in_lay2)
        #print("out_lay1", out_lay1.shape)
        
        in_lay3 = self.relu(out_lay2)
        out_lay3 = self.l_linear3(in_lay3)
        #print("in_lay3", in_lay3.shape)
        
        in_lay4 = self.relu(out_lay3)
        out = self.soft(self.l_linear4(in_lay4))
        #print("out", out.shape)
        
        #print(out)
        return out.squeeze()

In [ ]:
l_lstm1 = torch.nn.LSTM(input_size = 150, 
                             hidden_size = 150,
                             num_layers = 1,
                             batch_first = True)

l_lstm2 = torch.nn.LSTM(input_size = 150, 
                             hidden_size = 150,
                             num_layers = 1,
                             batch_first = True)

In [ ]:
#def train_sport_pred_2LSTM(lstm, train_loader, test_loader, learn_rate, epochs):
criterion1 = torch.nn.MSELoss()
optimizer1 = torch.optim.SGD(lstm.parameters(), lr = 1e-3)

criterion2 = torch.nn.MSELoss()
optimizer2 = torch.optim.SGD(lstm.parameters(), lr = 1e-3)
    
l_lstm1.eval()
accur = 0
#for step, ((input1, input2), result, (next_feat1, next_feat2)) in enumerate(test_loader):
 #   pred1 = l_lstm1(input1)
  #  pred2 = l_lstm2(input2)
        #print(pred.shape)
    #pred = torch.argmax(pred, dim = 1)
        #print(pred.shape)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3)
   # accur += pred.eq(next_feat1).sum().item()
#print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")
    
for epoch in range(15):
    losses_val1 = []
    losses_val2 = []
    accuracies = []
        
    l_lstm1.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), result, (next_feat1, next_feat2)) in enumerate(train_loader):
        lstm.zero_grad()
        pred1, (_,_) = l_lstm1(input1.float())
        pred1 = pred1[:,-1,:]
        pred2, (_,_) = l_lstm2(input2.float())
        pred2 = pred2[:,-1,:]
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
            #print(f"pred {pred.dtype}, result {result.dtype}")
        #print(f"pred1 {type(pred1)}, next feature {type(next_feat1)}")
        loss1 = criterion1(pred1, next_feat1.float())
        loss2 = criterion2(pred2, next_feat2.float())
        loss1.backward()
        optimizer1.step()
        
        loss2.backward()
        optimizer2.step()
        
    l_lstm1.eval()
    loss1 = 0
    loss2 = 0
    accur = 0
    for step, ((input1, input2), result, (next_feat1, next_feat2)) in enumerate(test_loader):
        pred1, (_,_) = l_lstm1(input1.float())
        pred1 = pred1[:,-1,:]
        
        pred2, (_,_) = l_lstm2(input2.float())
        pred2 = pred2[:,-1,:]
        #result_oh = result.to(torch.int64)
            #print(result.shape)
        #result_oh = torch.nn.functional.one_hot(result_oh, num_classes = 3)
            #print(result.shape)
            #print(pred.shape)
        loss1 += criterion1(pred1, next_feat1.float()).item()
        loss2 += criterion2(pred2, next_feat2.float()).item()
        #pred = torch.argmax(pred, dim = 1)
            #print(pred.shape)
            #print(pred.eq(result).sum())
        #accur += pred.eq(next_feat1).sum().item()
    losses_val1.append(loss1)
    losses_val2.append(loss2)
    accuracy = 100 * (accur /((step + 1) * 32))
    accuracies.append(accuracy)
    print(f"Loss1 {loss1} Loss2 {loss2}%") 
                
         

In [ ]:
contexts = []
for club in clubs.keys():
    a = clubs[club]
    #print(type(a))
    for _, games in a.iterrows():
        res_pos_neg = (int(games.possession_general_poss), int(games.fbref_own), int(games.fbref_oppon))
        contexts.append(res_pos_neg)
#        if int(games.result) == new_data.return_dicts("result")["W"]:
 #           res_pos_neg = (int(games.result), int(games.fbref_own), int(games.fbref_oppon))
  #          contexts.append(res_pos_neg)
   #     elif int(games.result) == new_data.return_dicts("result")["L"]:
    #        res_pos_neg = (int(games.result), int(games.fbref_oppon), int(games.fbref_own))
     #       contexts.append(res_pos_neg)
      #  else:
       #     res_pos_neg = (int(games.result), int(games.fbref_own), int(games.fbref_oppon))
        #    contexts.append(res_pos_neg)
         #   res_pos_neg = (int(games.result), int(games.fbref_oppon), int(games.fbref_own))
          #  contexts.append(res_pos_neg)

In [ ]:
pd.get_dummies(new_data_test.formation)

In [ ]:
new_data_test.data_frame.misc_performance_crdr

In [ ]:
class data_for_embed():

    def __init__(self, mylist):
        self.output = mylist
        

    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.output[idx]

In [ ]:
context_dataset = data_for_embed(contexts)

In [ ]:
def collate_data(pos_neg):
    result = torch.tensor([pos_neg[lab][0] for lab in range(len(pos_neg))], dtype=torch.int64)  
    context_positive = torch.tensor([pos_neg[lab][1] for lab in range(len(pos_neg))], dtype=torch.int64) 
    context_negative = torch.tensor([pos_neg[lab][2] for lab in range(len(pos_neg))], dtype=torch.int64) 
    return (result, context_positive, context_negative)

In [ ]:
context_dataset.__len__()

In [ ]:
class fast_word2vec(torch.nn.Module):
    def __init__(self, size, dims):
        super(fast_word2vec, self).__init__()
        self.vocab_size = size
        self.embed_dim = dims
        
        
        self.cent_embed = torch.nn.Embedding(self.vocab_size, self.embed_dim)
        self.cont_embed = torch.nn.Embedding(self.vocab_size, self.embed_dim)
        
        # Initialization
        self.cent_embed.weight.data.uniform_(-1, 1)
        self.cont_embed.weight.data.uniform_(-1, 1)
        
    def forward(self, sample):
        cent_word = sample[0]
        pos_con = sample[1]
        neg_con = sample[2]
        
        emb_cent = self.cent_embed(cent_word)
        emb_cont_pos = self.cont_embed(pos_con)
        emb_cont_neg = self.cont_embed(neg_con)

        pos = torch.clamp(torch.sum(emb_cent*emb_cont_pos, 1),max = 10, min = -10)
        neg = torch.clamp(torch.sum(emb_cent*emb_cont_neg, 1),max = 10, min = -10)
        
        m = torch.nn.Sigmoid()
        pos_score = torch.mean(m(pos), 0, True)
        neg_score = torch.mean(m(-neg), 0, True)
        
        #print(pos_score)
        #print(neg_score)
        return torch.neg(torch.cat((pos_score, neg_score), -1))

In [ ]:
# setting for training plus dataloader object
max_epochs = 300
lrn_rate = 0.05
loss_fn = torch.nn.NLLLoss()
club_len = len(set(list(new_data1.fbref_home_id) + list(new_data1.fbref_away_id)))
emb_dim = min(50, (club_len+1)/2)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
net = fast_word2vec(club_len, emb_dim).to(device) 

batch_size = 100
train_ldr = torch.utils.data.DataLoader(context_dataset, batch_size = batch_size, shuffle = True, collate_fn = collate_data)


In [ ]:
# function for training of the network
def train(net, ldr, bs, me, lr, loss_fn):
  # network, loader, batch_size, max_epochs, lerning_rate
    running_loss = []
    
    #loss_fn = nn.NLLLoss()
    
    target = torch.tensor([1, 0])
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr = lr)
    loss = net  
    print("\nStarting training")
    for itera in range(me):
        epoch_loss = 0.0
        
        for bix, cont in enumerate(ldr):
            opt.zero_grad()
            output = net.forward(cont)  
            loss = loss_fn(output, target)
            loss.backward()
            epoch_loss += loss.item()
            opt.step()
            
        if itera % 2 == 0:
            print("epoch = %4d   loss = %0.8f" % (itera, epoch_loss))
        running_loss.append(epoch_loss)
    print("Done")

In [ ]:
print("\nbatch_size = %3d " % batch_size)
print("max epochs = " + str(max_epochs))
print("loss = Sigmoid")
print("optimizer = Adam")
print("lrn_rate = %0.3f " % lrn_rate)
train(net, train_ldr, batch_size, 50, lrn_rate, loss_fn)

In [ ]:
import database_server.db_utilities as dbu

In [ ]:
df = dbu.select_query(query_str= "SELECT name, fbref_id FROM teams;")

In [ ]:
df_matches

In [ ]:
df["own_id"] = df.fbref_id.map(new_data.return_dicts("teams"))

In [ ]:
df[df.name == 'Bayern Munich']

In [ ]:
list(df.name)

In [ ]:
W, WT = net.parameters()
W = W.detach().numpy()

In [ ]:
W.shape

In [ ]:
def find_complementary_items(word, wordvecs, metric = "cosine", k = 5):
    # word or embedding, parameters form training, metric to find, num of comp items to find
    if type(word) == str:
        dist_matrix = distance.squareform(distance.pdist(wordvecs, metric))
        #index = vocab_trch[word]
        dists = dist_matrix[index]
        
    elif type(word) == np.ndarray:
        word = np.reshape(word, (1, 300), order = "F")
        wordvecs = np.concatenate((wordvecs,word), axis = 0)
        dist_matrix = distance.squareform(distance.pdist(wordvecs, metric))
        index = len(wordvecs)
        dists = dist_matrix[-1]
    ind = np.argpartition(dists, k)[:k+1]
    ind = ind[np.argsort(dists[ind])][1:]
    out = [(i, translator[vocab_i2t[i]], dists[i]) for i in ind]
    return out

In [ ]:
dist_matrix = distance.squareform(distance.pdist(W, "cosine"))
dists = dist_matrix[73]

ind = np.argpartition(dists, 5)[:5+1]
ind = ind[np.argsort(dists[ind])][1:]
out = [(i, df[df.own_id == i], dists[i]) for i in ind]

In [ ]:
from scipy.spatial import distance

In [ ]:
out

In [ ]:
from scraping.MarketValues import MarketValues

In [ ]:
df.name

In [ ]:
uniq_club = [x.lower().replace(" ","") for x in list(df.name)]
uniq_club

In [ ]:
#bl = MarketValues("bundesliga", 6)
#pl = MarketValues("premier league", 6)
#l1 = MarketValues("ligue 1", 6)
ll = MarketValues("la liga", 6)
sa = MarketValues("serie a", 6)

In [ ]:
df_bl = bl.get_club_overview()
df_pl = pl.get_club_overview()
df_l1 = l1.get_club_overview()
df_ll = ll.get_club_overview()
df_sa = sa.get_club_overview()

In [ ]:
uniq_club = [x.lower().replace(" ","") for x in list(df.name.unique())]
uniq_club

In [ ]:
unique_club_list = list(df_bl.club.unique()) + list(df_pl.club.unique()) + list(df_l1.club.unique()) + list(df_ll.club.unique()) + list(df_sa.club.unique())
unique_club_list

In [ ]:
import difflib

def similar(seq1, seq2):
    return difflib.SequenceMatcher(a=seq1.lower(), b=seq2.lower()).ratio()

In [ ]:
club_dict = {}
for club1 in uniq_club:
    previous = 0
    for club2 in unique_club_list:
        simil = similar(club1,club2)
        if simil > previous and club1 in club2:
            club_dict[club1] = club2
            previous = simil
        else:
            pass

club_dict

In [ ]:
print(np.setdiff1d(uniq_club,list(club_dict.keys())))
print(np.setdiff1d(unique_club_list,list(club_dict.values())))

In [ ]:
new_dict = {}       
for club3 in np.setdiff1d(uniq_club,list(club_dict.keys())):
    previous = 0
    for club4 in np.setdiff1d(unique_club_list,list(club_dict.values())):
        simil = similar(club3,club4)
        if simil > previous:
            new_dict[club3] = club4
            previous = simil
        else:
            pass
new_dict

In [ ]:
new_dict['heidenheim'] = None
new_dict['holsteinkiel'] = None
new_dict['nice'] = 'ogcnizza'
new_dict['napoli'] = "sscneapel"

In [ ]:
club_dict.update(new_dict)

In [ ]:
df.name = df.name.apply(lambda x: x.lower().replace(" ",""))

In [ ]:
df["trans_name"] = df.name.map(club_dict)

In [ ]:
df


In [ ]:
date_dict1 = new_data.return_dicts("season")
date_dict1_reverse = {v:k for k,v in date_dict1.items()}

date_dict2 = {}
i = 0
for date in df_bl.season.unique():
    date_dict2[date] = date_dict1_reverse[i]
    i+= 1
date_dict2

In [ ]:
date_dict1_reverse

In [ ]:
df_matches = dbu.select_query(query_str= "SELECT * FROM matches;")
df_matches

In [ ]:
for club in clubs.keys():
    df = clubs[club]
    df.head()
    break 

In [ ]:
df.head()

In [ ]:
new_data. 

In [ ]:
df["market_worth"] = 

In [ ]:
df["fbref_id_ds"] = df.fbref_id.map(new_data.return_dicts("teams"))
df

In [ ]:
df_bl

In [ ]:
next_feat1.shape

In [ ]:
l_lstm1(input1.float())[0]

In [ ]:
from numpy import unique
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
#from keras.layers.merge import concatenate
from keras.utils import plot_model

In [ ]:
pip install keras.layers #--upgrade pip

In [ ]:
# example of learned embedding encoding for a neural network


# load the dataset
def load_dataset(filename):
    # load the dataset as a pandas DataFrame
    data = read_csv(filename, header=None)
    # retrieve numpy array
    dataset = data.values
    # split into input (X) and output (y) variables
    X = dataset[:, :-1]
    y = dataset[:,-1]
    # format all fields as string
    X = X.astype(str)
    # reshape target to be a 2d array
    y = y.reshape((len(y), 1))
    return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
    X_train_enc, X_test_enc = list(), list()
    # label encode each column
    for i in range(X_train.shape[1]):
        le = LabelEncoder()
        le.fit(X_train[:, i])
        # encode
        train_enc = le.transform(X_train[:, i])
        test_enc = le.transform(X_test[:, i])
        # store
        X_train_enc.append(train_enc)
        X_test_enc.append(test_enc)
    return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

# load the dataset
X, y = load_dataset('breast-cancer.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# make output 3d
y_train_enc = y_train_enc.reshape((len(y_train_enc), 1, 1))
y_test_enc = y_test_enc.reshape((len(y_test_enc), 1, 1))
# prepare each input head
in_layers = list()
em_layers = list()
for i in range(len(X_train_enc)):
    # calculate the number of unique inputs
    n_labels = len(unique(X_train_enc[i]))
    # define input layer
    in_layer = Input(shape=(1,))
    # define embedding layer
    em_layer = Embedding(n_labels, 10)(in_layer)
    # store layers
    in_layers.append(in_layer)
    em_layers.append(em_layer)
# concat all embeddings
merge = concatenate(em_layers)
dense = Dense(10, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=in_layers, outputs=output)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# plot graph
plot_model(model, show_shapes=True, to_file='embeddings.png')
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=20, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
embedding_size = min(50,103)
#model = Model()
model = keras.models.Sequential()
model.add(Embedding(input_dim = 12, output_dim = embedding_size, input_length = 1, name="embedding"))
model.add(keras.layers.Flatten())
model.add(Dense(50, activation="relu"))
model.add(Dense(15, activation="relu"))
model.add(Dense(1))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["custom_metrics"])
model.fit(x = new_data1['referee'].values, y=new_data1['result_home'].values , epochs = 50, batch_size = 4)


In [ ]:
type(new_data1['referee'].values)

In [ ]:
new_data1.referee.as_matrix()

In [ ]:
import keras

In [ ]:
class Sport_pred_LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features
        self.num_classes = num_classes
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers)

        self.l_linear = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.l_linear2 = torch.nn.Linear(self.n_hidden, num_classes)
        self.soft = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()
        
    #def init_hidden(self, batch_size):
     #   hidden_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
      #  cell_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
       # self.hidden = (hidden_state, cell_state)


    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        #print(x)
        h0 = torch.randn(self.n_layers*1, 38, self.n_hidden).to(x.dtype)
        #print(h0.dtype)
        c0 = torch.randn(self.n_layers*1, 38, self.n_hidden).to(x.dtype)
        #print(c0.dtype)
        
        lstm_out,(hn, cn) = self.l_lstm(x,(h0, c0))
        #print("lstm out:", lstm_out)
        #x = lstm_out.contiguous().view(-1, self.n_hidden)
        #x = lstm_out[:, -1, :]
        print(hn.shape)
        x = hn.view(-1, self.n_hidden)
        print(x.shape)
        x = self.relu(self.l_linear(x))
        print(x.shape)
        out = self.soft(self.l_linear2(x))
        print(out.shape)
        return out

In [ ]:
for _, (s,p) in enumerate(train_loader):
    print(s.shape)
    print(p.shape)
    break

In [ ]:
class Sport_pred_LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features
        self.num_classes = num_classes
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers)

        self.l_linear = torch.nn.Linear(self.n_hidden, num_classes)
        self.soft = torch.nn.Softmax(dim = 1)
        
    #def init_hidden(self, batch_size):
     #   hidden_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
      #  cell_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
       # self.hidden = (hidden_state, cell_state)


    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        #print(x)
        h0 = torch.randn(self.n_layers*1, self.n_hidden).to(x.dtype)
        #print(h0.dtype)
        c0 = torch.randn(self.n_layers*1, self.n_hidden).to(x.dtype)
        #print(c0.dtype)
        
        lstm_out,_ = self.l_lstm(x,(h0, c0))
        #print("lstm out:", lstm_out)
        x = lstm_out.contiguous().view(-1, self.n_hidden)
        #print(x)
        out = self.soft(self.l_linear(x))
        #print(out)
        return out

In [ ]:
n_features = len(X1[0]) # this is number of parallel inputs

# create NN
mv_net = Sport_pred_LSTM(n_features, 150, 3)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-4, weight_decay = 1e-3)

train_episodes = 500
batch_size = 30





In [ ]:
mv_net.train()

for t in range(train_episodes):

    mv_net.train()
    for _, (data, target) in enumerate(dataloader):
        inpt = data.to(torch.float32)
        label = target
        #print(data.dtype)

        output = mv_net.forward(data) 
        print(output)
        loss = criterion(output, label.to(torch.float32)) 
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
        break
    if t%50 == 0:
        print('step : ' , t , 'loss : ' , loss.item())
        
    #mv_net.eval()
    

In [ ]:
df.fbref_match_id

In [ ]:

def sequencizer(X, y, seq_len):
    x_t = []
    y_t = []
    for i in range(len(X)):
        if i + seq_len + 1 == len(X):
            break
        x_train = X[i:i + seq_len]
        y_train = y[i + seq_len + 1]
        
        x_t.append(x_train)
        y_t.append(y_train)
        
    return np.array(x_t), np.array(y_t)

    

In [ ]:
new_data1

In [ ]:
empty_df = pd.DataFrame(columns = list(new_data.dataset_team(0)), index = [0])

for team in set(list(new_data1.fbref_home_id.unique()) + list(new_data1.fbref_away_id.unique())):
    empty_df = pd.concat([new_data.dataset_team(team), empty_df])

In [ ]:
dts2.loc["2023-04-25"]

In [ ]:
dts1#.loc["2022-08-13"]

In [ ]:
df = new_data.data_frame
X = []
y = []
for team in set(list(empty_df.fbref_own.unique()) + list(empty_df.fbref_oppon.unique())):
    for seas in empty_df.fbref_season.unique():
        dts1 = empty_df[(empty_df.fbref_own == team) & (empty_df.fbref_season == seas)]
        for i in range(len(dts1)):
            i += 11
            date11 = dts1.index[i - 11]
            date12 = dts1.index[i - 1]
            
            dts2 = empty_df[(empty_df.fbref_own == dts1.iloc[i].fbref_oppon) & (empty_df.fbref_season == seas)]
            date21 = dts2.index[i - 11]
            date22 = dts2.index[i - 1]
            
            #if dts1.index[i] != dts2.index[i]:
             #   print(dts1.iloc[i])
              #  print(dts2.iloc[i])
            if dts1.iloc[i].fbref_match_id != dts2.iloc[i].fbref_match_id:
                print("Alarm")
            x = [np.array(dts1.loc[date11:date12]), np.array(dts2.loc[date21:date22])]
            X.append(x)
            y.append(dts1.iloc[i].result) 
            break
            if i == len(dts1)-1:
                break
    print(team)

In [ ]:
empty_df[(empty_df.fbref_own == team) & (empty_df.fbref_season == seas)].index[10]

In [ ]:
X = [torch.from_numpy(x.astype(float)) for z in X for x in z]
y = [torch.from_numpy(Y.astype(float)) for Y in y]
y = [torch.nn.functional.one_hot(x.long(), num_classes = 3) for x in y]

In [ ]:
class data_to_2lstm():

    def __init__(self, mylist1, mylist2):
        self.output =[]
        
        for i in range(len(mylist1)):
            self.output.append((mylist1[i][0], mylist1[i][1], mylist2[i]))
        
 

    def __len__(self):
        return len(self.output1)

    def __getitem__(self, idx):
        return self.output[idx]


In [ ]:
dt2lstm = data_to_2lstm(X,y)
dataloader = DataLoader(dt2lstm, batch_size = 30,
                        shuffle = False)
dataloader

In [ ]:
dt2lstm.__getitem__(0)

In [ ]:
for i, (x1, x2, y) in enumerate(dataloader):
    print(i, x1, y)
    break

In [ ]:

X = {}
y = []

#set(list(df.fbref_away_id.unique()) + list(df.fbref_home_id.unique()))
for team in set(list(df.fbref_away_id.unique()) + list(df.fbref_home_id.unique())):
    data_team = new_data.dataset_team(team)
    for seas in data_team.fbref_season.unique():
        dts = data_team[data_team.fbref_season == seas]
        for i in range(len(dts)):
            string = str(team) + "_" + str(seas) + "_"
            X.append(dts.iloc[i:i + 10])
            y.append(dts.iloc[i + 10 + 1].result)
            #print(dts.iloc[i:i+10].result)
            if i + 10 + 1 == len(dts)-1:
                break



In [ ]:
X[0].shape

In [ ]:
#dts.iloc[0:4]
X = []
y = []
for i in range(len(dts)):
    X.append(dts.iloc[i:i + 10])
    y.append(dts.iloc[i + 10 + 1])
    #print(dts.iloc[i:i+10].result)
    if i + 10 + 1 == len(dts)-1:
        break

In [ ]:
empty_df

In [ ]:
np.array(X[0])
#X[0]

In [ ]:
class Sport_pred_standard_nn(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(self.n_features * 2, self.n_hidden * 2)
        self.l_linear2 = torch.nn.Linear(self.n_hidden * 2, self.n_hidden * 0.5)
        self.l_linear3 = torch.nn.Linear(self.n_hidden * 0.5, num_classes)
        
        self.soft = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        out_lay1 = self.l_linear1(in_lay1)
        in_lay2 = self.relu(out_lay1)
        out_lay2 = self.l_linear2(in_lay2)
        in_lay3 = self.relu(out_lay3)
        out = self.soft(self.l_linear3(in_lay3))

        return out

In [ ]:
class PositionalEncoder(torch.nn.Module):

    def __init__(self, dropout: float = 0.1, max_seq_len: int = 5000, d_model: int = 512, batch_first: bool = False):

        """
        Parameters:
            dropout: the dropout rate
            max_seq_len: the maximum length of the input sequences
            d_model: The dimension of the output of sub-layers in the model 
                     (Vaswani et al, 2017)
        """

        super().__init__()

        self.d_model = d_model
        
        self.dropout = nn.Dropout(p=dropout)

        self.batch_first = batch_first

        self.x_dim = 1 if batch_first else 0

        # copy pasted from PyTorch tutorial
        position = torch.arange(max_seq_len).unsqueeze(1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        
        pe = torch.zeros(max_seq_len, 1, d_model)
        
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, enc_seq_len, dim_val] or 
               [enc_seq_len, batch_size, dim_val]
        """

        x = x + self.pe[:x.size(self.x_dim)]

        return self.dropout(x)


In [ ]:
class Sport_pred_Transformer(torch.nn.Module):

    def __init__(self, input_size: int, dec_seq_len: int, batch_first: bool, out_seq_len: int = 58, 
                 dim_val: int = 512, n_encoder_layers: int = 4, n_decoder_layers: int = 4, n_heads: int = 8,
                 dropout_encoder: float = 0.2, dropout_decoder: float = 0.2, dropout_pos_enc: float = 0.1,
                 dim_feedforward_encoder: int = 2048, dim_feedforward_decoder: int = 2048, 
                 num_predicted_features: int = 1): 

        """
        Args:
            input_size: int, number of input variables. 1 if univariate.
            dec_seq_len: int, the length of the input sequence fed to the decoder
            dim_val: int, aka d_model. All sub-layers in the model produce 
                     outputs of dimension dim_val
            n_encoder_layers: int, number of stacked encoder layers in the encoder
            n_decoder_layers: int, number of stacked encoder layers in the decoder
            n_heads: int, the number of attention heads (aka parallel attention layers)
            dropout_encoder: float, the dropout rate of the encoder
            dropout_decoder: float, the dropout rate of the decoder
            dropout_pos_enc: float, the dropout rate of the positional encoder
            dim_feedforward_encoder: int, number of neurons in the linear layer 
                                     of the encoder
            dim_feedforward_decoder: int, number of neurons in the linear layer 
                                     of the decoder
            num_predicted_features: int, the number of features you want to predict.
                                    Most of the time, this will be 1 because we're
                                    only forecasting FCR-N prices in DK2, but in
                                    we wanted to also predict FCR-D with the same
                                    model, num_predicted_features should be 2.
        """

        super().__init__() 

        self.dec_seq_len = dec_seq_len

        #print("input_size is: {}".format(input_size))
        #print("dim_val is: {}".format(dim_val))

        # Creating the three linear layers needed for the model
        self.encoder_input_layer = nn.Linear(
            in_features = input_size, 
            out_features = dim_val 
            )

        self.decoder_input_layer = nn.Linear(
            in_features = num_predicted_features,
            out_features = dim_val
            )  
        
        self.linear_mapping = nn.Linear(
            in_features = dim_val, 
            out_features = num_predicted_features
            )

        # Create positional encoder
        self.positional_encoding_layer = pe.PositionalEncoder(
            d_model = dim_val,
            dropout = dropout_pos_enc
            )

        # The encoder layer used in the paper is identical to the one used by
        # Vaswani et al (2017) on which the PyTorch module is based.
        encoder_layer = nn.TransformerEncoderLayer(
            d_model = dim_val, 
            nhead = n_heads,
            dim_feedforward = dim_feedforward_encoder,
            dropout = dropout_encoder,
            batch_first = batch_first
            )

        # Stack the encoder layers in nn.TransformerDecoder
        # It seems the option of passing a normalization instance is redundant
        # in my case, because nn.TransformerEncoderLayer per default normalizes
        # after each sub-layer
        # (https://github.com/pytorch/pytorch/issues/24930).
        self.encoder = nn.TransformerEncoder(
            encoder_layer = encoder_layer,
            num_layers = n_encoder_layers, 
            norm = None
            )

        decoder_layer = nn.TransformerDecoderLayer(
            d_model = dim_val,
            nhead = n_heads,
            dim_feedforward = dim_feedforward_decoder,
            dropout = dropout_decoder,
            batch_first = batch_first
            )

        # Stack the decoder layers in nn.TransformerDecoder
        # It seems the option of passing a normalization instance is redundant
        # in my case, because nn.TransformerDecoderLayer per default normalizes
        # after each sub-layer
        # (https://github.com/pytorch/pytorch/issues/24930).
        self.decoder = nn.TransformerDecoder(
            decoder_layer = decoder_layer,
            num_layers = n_decoder_layers, 
            norm = None
            )

    def forward(self, src: torch.Tensor, tgt: torch.Tensor, src_mask: torch.Tensor = None, 
                tgt_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Returns a tensor of shape:
        [target_sequence_length, batch_size, num_predicted_features]
        
        Args:
            src: the encoder's output sequence. Shape: (S,E) for unbatched input, 
                 (S, N, E) if batch_first=False or (N, S, E) if 
                 batch_first=True, where S is the source sequence length, 
                 N is the batch size, and E is the number of features (1 if univariate)
            tgt: the sequence to the decoder. Shape: (T,E) for unbatched input, 
                 (T, N, E)(T,N,E) if batch_first=False or (N, T, E) if 
                 batch_first=True, where T is the target sequence length, 
                 N is the batch size, and E is the number of features (1 if univariate)
            src_mask: the mask for the src sequence to prevent the model from 
                      using data points from the target sequence
            tgt_mask: the mask for the tgt sequence to prevent the model from
                      using data points from the target sequence
        """

        #print("From model.forward(): Size of src as given to forward(): {}".format(src.size()))
        #print("From model.forward(): tgt size = {}".format(tgt.size()))

        # Pass throguh the input layer right before the encoder
        src = self.encoder_input_layer(src) # src shape: [batch_size, src length, dim_val] regardless of number of input features
        #print("From model.forward(): Size of src after input layer: {}".format(src.size()))

        # Pass through the positional encoding layer
        src = self.positional_encoding_layer(src) # src shape: [batch_size, src length, dim_val] regardless of number of input features
        #print("From model.forward(): Size of src after pos_enc layer: {}".format(src.size()))

        # Pass through all the stacked encoder layers in the encoder
        # Masking is only needed in the encoder if input sequences are padded
        # which they are not in this time series use case, because all my
        # input sequences are naturally of the same length. 
        # (https://github.com/huggingface/transformers/issues/4083)
        src = self.encoder( # src shape: [batch_size, enc_seq_len, dim_val]
            src=src
            )
        #print("From model.forward(): Size of src after encoder: {}".format(src.size()))

        # Pass decoder input through decoder input layer
        decoder_output = self.decoder_input_layer(tgt) # src shape: [target sequence length, batch_size, dim_val] regardless of number of input features
        #print("From model.forward(): Size of decoder_output after linear decoder layer: {}".format(decoder_output.size()))

        #if src_mask is not None:
            #print("From model.forward(): Size of src_mask: {}".format(src_mask.size()))
        #if tgt_mask is not None:
            #print("From model.forward(): Size of tgt_mask: {}".format(tgt_mask.size()))

        # Pass throguh decoder - output shape: [batch_size, target seq len, dim_val]
        decoder_output = self.decoder(
            tgt=decoder_output,
            memory=src,
            tgt_mask=tgt_mask,
            memory_mask=src_mask
            )

        #print("From model.forward(): decoder_output shape after decoder: {}".format(decoder_output.shape))

        # Pass through linear mapping
        decoder_output = self.linear_mapping(decoder_output) # shape [batch_size, target seq len]
        #print("From model.forward(): decoder_output size after linear_mapping = {}".format(decoder_output.size()))

        return decoder_output


In [ ]:
## Model parameters
dim_val = 512 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_decoder_layers = 4 # Number of times the decoder layer is stacked in the decoder
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 1 # The number of input variables. 1 if univariate forecasting.
dec_seq_len = 92 # length of input given to decoder. Can have any integer value.
enc_seq_len = 153 # length of input given to encoder. Can have any integer value.
output_sequence_length = 58 # Length of the target sequence, i.e. how many time steps should your forecast cover
max_seq_len = enc_seq_len # What's the longest sequence the model will encounter? Used to make the positional encoder

model = tst.TimeSeriesTransformer(
    dim_val=dim_val,
    input_size=input_size, 
    dec_seq_len=dec_seq_len,
    max_seq_len=max_seq_len,
    out_seq_len=output_sequence_length, 
    n_decoder_layers=n_decoder_layers,
    n_encoder_layers=n_encoder_layers,
    n_heads=n_heads)


In [ ]:

def get_src_trg(
        self,
        sequence: torch.Tensor, 
        enc_seq_len: int, 
        target_seq_len: int
        ) -> Tuple[torch.tensor, torch.tensor, torch.tensor]:

        """
        Generate the src (encoder input), trg (decoder input) and trg_y (the target)
        sequences from a sequence. 
        Args:
            sequence: tensor, a 1D tensor of length n where 
                    n = encoder input length + target sequence length  
            enc_seq_len: int, the desired length of the input to the transformer encoder
            target_seq_len: int, the desired length of the target sequence (the 
                            one against which the model output is compared)
        Return: 
            src: tensor, 1D, used as input to the transformer model
            trg: tensor, 1D, used as input to the transformer model
            trg_y: tensor, 1D, the target sequence against which the model output
                is compared when computing loss. 
        
        """
        #print("Called dataset.TransformerDataset.get_src_trg")
        assert len(sequence) == enc_seq_len + target_seq_len, "Sequence length does not equal (input length + target length)"
        
        #print("From data.TransformerDataset.get_src_trg: sequence shape: {}".format(sequence.shape))

        # encoder input
        src = sequence[:enc_seq_len] 
        
        # decoder input. As per the paper, it must have the same dimension as the 
        # target sequence, and it must contain the last value of src, and all
        # values of trg_y except the last (i.e. it must be shifted right by 1)
        trg = sequence[enc_seq_len-1:len(sequence)-1]

        #print("From data.TransformerDataset.get_src_trg: trg shape before slice: {}".format(trg.shape))

        trg = trg[:, 0]

        #print("From data.TransformerDataset.get_src_trg: trg shape after slice: {}".format(trg.shape))

        if len(trg.shape) == 1:

            trg = trg.unsqueeze(-1)

            #print("From data.TransformerDataset.get_src_trg: trg shape after unsqueeze: {}".format(trg.shape))

        
        assert len(trg) == target_seq_len, "Length of trg does not match target sequence length"

        # The target sequence against which the model output will be compared to compute loss
        trg_y = sequence[-target_seq_len:]

        #print("From data.TransformerDataset.get_src_trg: trg_y shape before slice: {}".format(trg_y.shape))

        # We only want trg_y to consist of the target variable not any potential exogenous variables
        trg_y = trg_y[:, 0]

        #print("From data.TransformerDataset.get_src_trg: trg_y shape after slice: {}".format(trg_y.shape))

        assert len(trg_y) == target_seq_len, "Length of trg_y does not match target sequence length"

        return src, trg, trg_y.squeeze(-1) # change size from [batch_size, target_seq_len, num_features] to [batch_size, target_seq_len] 


In [ ]:

def generate_square_subsequent_mask(dim1: int, dim2: int) -> Tensor:
    """
    Generates an upper-triangular matrix of -inf, with zeros on diag.
    Source:
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    Args:
        dim1: int, for both src and tgt masking, this must be target sequence
              length
        dim2: int, for src masking this must be encoder sequence length (i.e. 
              the length of the input sequence to the model), 
              and for tgt masking, this must be target sequence length 
    Return:
        A Tensor of shape [dim1, dim2]
    """
    return torch.triu(torch.ones(dim1, dim2) * float('-inf'), diagonal=1)

# Input length
enc_seq_len = 100

# Output length
output_sequence_length = 58

# Make src mask for decoder with size:
tgt_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=output_sequence_length
   )

src_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=enc_seq_len
    )


In [ ]:

output = model(
    src=src, 
    tgt=trg,
    src_mask=src_mask,
    tgt_mask=tgt_mask
    )

In [ ]:
for input_,target in dataloader:
    print(input_.shape)
    net = torch.nn.LSTM(288,10,1)
    h0 = torch.randn(1, 10)
    c0 = torch.randn(1, 10)
    output, (hn, cn) = net(input_, (h0, c0))
    break

In [ ]:
data.to(torch.float32)

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) # fully connected 
        self.fc_2 = nn.Linear(128, num_classes) # fully connected last layer
        self.relu = nn.ReLU()
        
    def forward(self,x):
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        return out

In [ ]:
def data_to_input(data_frame):
    club_np = data_frame.to_numpy()
    X = []
    y = []
    for cl in range(window_size, len(club_np)-1):
        x = [[a] for a in club_np[cl-window_size:cl,3:]]
        label = club_np[cl+1,0]
        X.append(x)
        y.append(label)
    return X,y
